In [ ]:
import pandas as pd
import json
import sqlite3
import os
import glob
from tqdm import tqdm

# Ana Proje Yolu (Senin Drive yapına göre)
BASE_PATH = '/content/drive/MyDrive/FSTurk'
os.chdir(BASE_PATH)

print(f"📂 Çalışma dizini: {os.getcwd()}")
print("✅ Gerekli kütüphaneler yüklendi.")

📂 Çalışma dizini: /content/drive/MyDrive/FSTurk
✅ Gerekli kütüphaneler yüklendi.


In [ ]:
#" ", -, . gibi karakterler için özelleşmiş ZEMBEREK VE KAIKKI FİLTRASYONU

import json
import os
import glob
from collections import defaultdict

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
ZEMBEREK_FILE = os.path.join(BASE_PATH, "turkish_processed_zemberek_lexicon.json")
KAIKKI_FOLDER = os.path.join(BASE_PATH, "kaikki_processing")
OUTPUT_FILE = os.path.join(BASE_PATH, "final_lexicon_atomic_base.json")

# TR -> EN MAPPING
TAG_MAPPING = {
    'ISIM': 'NOUN',     'NOUN': 'NOUN',
    'FIIL': 'VERB',     'VERB': 'VERB',
    'SIFAT': 'ADJ',     'ADJ': 'ADJ',
    'ZARF': 'ADV',      'ADV': 'ADV',
    'ZAMIR': 'PRON',    'PRON': 'PRON',
    'BAGLAC': 'CONJ',   'CONJ': 'CONJ', 'CCONJ': 'CONJ',
    'SAYI': 'NUM',      'NUM': 'NUM',
    'EDAT': 'ADP',      'ADP': 'ADP',
    'BELIRTEC': 'DET',  'DET': 'DET',
    'OZEL ISIM': 'NOUN', 'PROPN': 'NOUN',
    'UNK': 'OTHER',     'OTHER': 'OTHER'
}

def is_clean_atomic_word(word, tag):
    """
    Kelimenin FST için uygun, temiz ve atomik olup olmadığını kontrol eder.
    """
    word = word.strip()

    # 1. Boşluk Kontrolü (Multi-word)
    if ' ' in word: return False

    # 2. Tire Kontrolü (Compound words) -> "e-devlet" istemiyoruz
    if '-' in word: return False

    # 3. Uzunluk Kontrolü (Tek harf 'o' hariç genelde çöp olabilir ama şimdilik tutalım)
    if len(word) == 0: return False

    # 4. Sayı/Sembol Kontrolü (NUM kategorisi hariç)
    # Eğer kelime NUM değilse ama içinde rakam varsa (örn: "covid19") atalım mı?
    # Evet, temiz sözlük için atalım.
    if tag != 'NUM':
        # isalpha(): Sadece harf mi? (Türkçe karakterler ve şapkalar dahildir)
        # Ancak tire ve boşluk zaten yukarıda elendi.
        # Bazı kısaltmalar (T.C.) noktalı olabilir. FST genelde noktasız sever.
        if '.' in word: return False

        # İçinde rakam var mı? (Word123 -> Çöp)
        if any(char.isdigit() for char in word): return False

    return True

def create_pristine_lexicon():
    print("💎 PRISTINE (TERTEMİZ) SÖZLÜK OLUŞTURULUYOR...")
    print("   🚫 Filtreler: Boşluk, Tire (-), Nokta (.), Rakamlı Kelimeler")

    grouped_data = defaultdict(set)
    dropped_count = 0

    # --- 1. ZEMBEREK ---
    print(f"1️⃣ Zemberek taranıyor...")
    if os.path.exists(ZEMBEREK_FILE):
        with open(ZEMBEREK_FILE, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)

            # Veri listeyse
            if isinstance(raw_data, list):
                for item in raw_data:
                    if len(item) >= 2:
                        word = item[0]
                        tr_tag = item[1]
                        en_tag = TAG_MAPPING.get(tr_tag, 'OTHER')

                        if is_clean_atomic_word(word, en_tag):
                            grouped_data[en_tag].add(word.strip())
                        else:
                            dropped_count += 1

            # Veri sözlükse
            elif isinstance(raw_data, dict):
                for tag, words in raw_data.items():
                    en_tag = TAG_MAPPING.get(tag, 'OTHER')
                    for w in words:
                        if is_clean_atomic_word(w, en_tag):
                            grouped_data[en_tag].add(w.strip())
                        else:
                            dropped_count += 1

    # --- 2. KAIKKI ---
    print(f"2️⃣ Kaikki taranıyor...")
    jsonl_files = glob.glob(os.path.join(KAIKKI_FOLDER, "*.jsonl"))

    for filepath in jsonl_files:
        filename = os.path.basename(filepath)
        if filename == "tr-extract.jsonl": continue

        raw_tag = filename.replace('.jsonl', '')
        en_tag = TAG_MAPPING.get(raw_tag, raw_tag) # Varsayılan: dosya adı

        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    word = data.get('word')
                    if word:
                        if is_clean_atomic_word(word, en_tag):
                            grouped_data[en_tag].add(word.strip())
                        else:
                            dropped_count += 1
                except: continue

    # --- 3. KAYDETME ---
    print(f"3️⃣ Final dosya yazılıyor...")

    final_output = {}
    total_words = 0

    for tag, word_set in grouped_data.items():
        final_output[tag] = sorted(list(word_set))
        total_words += len(final_output[tag])
        print(f"   🔹 {tag}: {len(final_output[tag])} kelime")

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, ensure_ascii=False)

    print(f"\n✨ TERTEMİZ BAŞLANGIÇ HAZIR!")
    print(f"   🗑️ Filtrelenen (Çöp/Bileşik) Kelime: {dropped_count}")
    print(f"   💎 Saf Kelime Sayısı: {total_words}")
    print(f"   📂 Dosya: {OUTPUT_FILE}")

create_pristine_lexicon()

💎 PRISTINE (TERTEMİZ) SÖZLÜK OLUŞTURULUYOR...
   🚫 Filtreler: Boşluk, Tire (-), Nokta (.), Rakamlı Kelimeler
1️⃣ Zemberek taranıyor...
2️⃣ Kaikki taranıyor...
3️⃣ Final dosya yazılıyor...
   🔹 OTHER: 90758 kelime
   🔹 PRON: 135 kelime
   🔹 NOUN: 145538 kelime
   🔹 ADJ: 10831 kelime
   🔹 NUM: 11 kelime
   🔹 VERB: 68180 kelime
   🔹 CONJ: 52 kelime
   🔹 ADV: 1703 kelime

✨ TERTEMİZ BAŞLANGIÇ HAZIR!
   🗑️ Filtrelenen (Çöp/Bileşik) Kelime: 34980
   💎 Saf Kelime Sayısı: 317208
   📂 Dosya: /content/drive/MyDrive/FSTurk/final_lexicon_atomic_base.json


In [ ]:
#--AMAÇ WIKIMEDIA VERİLERİNİ STANZA için  TÜRLERİNE AYIRMA--
import json
import os
import re
from tqdm import tqdm

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
# Şu anki en güncel dosyan bu (Zemberek + Kaikki + Gruplanmış):
INPUT_LEXICON = os.path.join(BASE_PATH, "final_lexicon_grouped_english.json")
CORPUS_FILE = os.path.join(BASE_PATH, "wikimedia_data", "cleaned_corpus.txt")
OUTPUT_FILE = os.path.join(BASE_PATH, "contain_zemberek_kaikki_wikimedia_with_stanza.json")

def add_corpus_words_to_waiting_room():
    print("🚀 WIKIPEDIA TARAMASI BAŞLIYOR...")

    # 1. Mevcut Sözlüğü Yükle
    if not os.path.exists(INPUT_LEXICON):
        print("❌ Sözlük bulunamadı!")
        return

    print(f"1️⃣ Mevcut sözlük yükleniyor...")
    with open(INPUT_LEXICON, 'r', encoding='utf-8') as f:
        lexicon_data = json.load(f)

    # Hızlı kontrol için mevcut kelimelerin hepsini bir kümeye (set) alalım
    existing_words = set()
    for tag, word_list in lexicon_data.items():
        for w in word_list:
            existing_words.add(w)

    print(f"   ✅ Hafızada zaten {len(existing_words)} kelime var.")

    # 2. Corpus Dosyasını Tara
    print(f"2️⃣ Corpus dosyasından YENİ kelimeler aranıyor...")

    if not os.path.exists(CORPUS_FILE):
        print("❌ Corpus dosyası yok! Lütfen wikimedia_data klasörünü kontrol et.")
        return

    new_candidates = set()
    # Regex: Sadece Türkçe karakter içeren kelimeleri al (noktalama yok, sayı yok)
    word_pattern = re.compile(r"[a-zA-ZçÇğĞıİöÖşŞüÜ]+")

    with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
        # Tqdm ile ilerleme çubuğu göster
        for line in tqdm(f, desc="Wiki Taranıyor"):
            words = word_pattern.findall(line)
            for w in words:
                # Küçük harfe çevir (Lexicon standardı için)
                w_clean = w.lower()

                # Kurallar:
                # 1. 2 harften uzun olsun (ve, de, ki gibi bağlaçlar zaten vardır)
                # 2. Zaten sözlükte OLMASIN
                if len(w_clean) > 2 and w_clean not in existing_words:
                    new_candidates.add(w_clean)

    print(f"   ✨ Wikipedia'dan {len(new_candidates)} adet YEPYENİ kelime bulundu!")

    # 3. Bunları 'OTHER' Kategorisine Ekle (Bekleme Odası)
    if 'OTHER' not in lexicon_data:
        lexicon_data['OTHER'] = []

    # Kümeyi listeye çevirip ekle
    lexicon_data['OTHER'].extend(list(new_candidates))

    # OTHER içindeki tekrarları temizle
    lexicon_data['OTHER'] = sorted(list(set(lexicon_data['OTHER'])))

    print(f"   📦 Yeni kelimeler 'OTHER' (Formatlanacaklar) listesine eklendi.")
    print(f"   🧐 Şu an 'OTHER' toplam boyutu: {len(lexicon_data['OTHER'])}")

    # 4. Kaydet
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(lexicon_data, f, ensure_ascii=False)

    print(f"\n💾 Dosya Kaydedildi: {OUTPUT_FILE}")
    print("👉 SIRADAKİ ADIM: Stanza kodunu bu dosya üzerinde çalıştırıp 'OTHER'ları eritmek!")

add_corpus_words_to_waiting_room()

🚀 WIKIPEDIA TARAMASI BAŞLIYOR...
1️⃣ Mevcut sözlük yükleniyor...
   ✅ Hafızada zaten 322359 kelime var.
2️⃣ Corpus dosyasından YENİ kelimeler aranıyor...


Wiki Taranıyor: 100000it [00:35, 2812.30it/s]


   ✨ Wikipedia'dan 1033121 adet YEPYENİ kelime bulundu!
   📦 Yeni kelimeler 'OTHER' (Formatlanacaklar) listesine eklendi.
   🧐 Şu an 'OTHER' toplam boyutu: 1129886

💾 Dosya Kaydedildi: /content/drive/MyDrive/FSTurk/contain_zemberek_kaikki_wikimedia_with_stanza.json
👉 SIRADAKİ ADIM: Stanza kodunu bu dosya üzerinde çalıştırıp 'OTHER'ları eritmek!


In [ ]:
#T4 SEÇTİĞİMDEN EMİN OLUN.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install stanza

In [1]:
import torch
print("GPU Var mı?:", torch.cuda.is_available())
try:
    print("GPU Adı:", torch.cuda.get_device_name(0))
    print("✅ GPU AKTİF! Devam edebilirsin.")
except:
    print("❌ GPU YOK! Ayarları kontrol et.")

GPU Var mı?: True
GPU Adı: Tesla T4
✅ GPU AKTİF! Devam edebilirsin.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 43.8 MB/s eta 0:00:00


In [4]:
import stanza
import json
import os
from tqdm import tqdm
import math
import torch

# --- AYARLAR ---
BASE_PATH = '/content/drive/MyDrive/FSTurk'
INPUT_FILE = os.path.join(BASE_PATH, "contain_zemberek_kaikki_wikimedia_with_stanza.json")
OUTPUT_FILE = os.path.join(BASE_PATH, "MASTER_FINAL_LEXICON.json")
TEMP_SAVE_FILE = os.path.join(BASE_PATH, "temp_stanza_progress.json")

def turbo_stanza_safe():
    # 1. GPU ZORUNLU KONTROL
    if not torch.cuda.is_available():
        raise Exception("❌ HATA: GPU algılanamadı! Runtime ayarlarından T4 GPU seçili mi?")

    print("✅ GPU Algılandı, Turbo Mod Başlatılıyor...")

    # 2. Stanza Yükle
    stanza.download('tr', processors='tokenize,pos', verbose=False)
    # use_gpu=True ZORUNLU
    nlp = stanza.Pipeline('tr', processors='tokenize,pos', use_gpu=True, verbose=False, tokenize_no_ssplit=True)

    # 3. Veri Yükle
    if not os.path.exists(INPUT_FILE):
        print("❌ Dosya bulunamadı!")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    full_other_list = data.get('OTHER', [])
    print(f"   📦 Toplam 'OTHER' Kelime Sayısı: {len(full_other_list)}")

    if len(full_other_list) == 0:
        print("   ✅ İşlenecek veri yok!")
        return

    # 4. Checkpoint Kontrolü
    processed_results = {}

    if os.path.exists(TEMP_SAVE_FILE):
        print("   📂 Kayıtlı ilerleme bulundu! Devam ediliyor...")
        with open(TEMP_SAVE_FILE, 'r', encoding='utf-8') as f:
            processed_results = json.load(f)

            # Zaten yapılanları küme (set) içine alıp hızla eleyelim
            done_set = set()
            for v_list in processed_results.values():
                done_set.update(v_list)

            # List comprehension ile filtrele
            full_other_list = [w for w in full_other_list if w not in done_set]
            print(f"   ⏩ {len(done_set)} kelime atlandı. Kalan: {len(full_other_list)}")

    else:
        processed_results = {
            'NOUN': [], 'VERB': [], 'ADJ': [], 'ADV': [],
            'PRON': [], 'CONJ': [], 'NUM': [], 'ADP': [], 'DET': []
        }

    # 5. BATCH PROCESSING
    BATCH_SIZE = 500
    total_batches = math.ceil(len(full_other_list) / BATCH_SIZE)

    print("   🔥 İşlem Başlıyor...")
    save_counter = 0

    # Tqdm ile ilerle
    for i in tqdm(range(0, len(full_other_list), BATCH_SIZE), total=total_batches, desc="Turbo GPU"):
        batch_words = full_other_list[i : i + BATCH_SIZE]

        # Kelimeleri birleştir (Stanza için tek metin)
        batch_text = "\n".join(batch_words)

        try:
            doc = nlp(batch_text)

            for sentence in doc.sentences:
                for word_obj in sentence.words:
                    if word_obj.upos in ['NOUN', 'PROPN']: t = 'NOUN'
                    elif word_obj.upos == 'VERB': t = 'VERB'
                    elif word_obj.upos == 'ADJ': t = 'ADJ'
                    elif word_obj.upos == 'ADV': t = 'ADV'
                    elif word_obj.upos == 'PRON': t = 'PRON'
                    elif word_obj.upos in ['CCONJ', 'SCONJ']: t = 'CONJ'
                    elif word_obj.upos == 'NUM': t = 'NUM'
                    elif word_obj.upos == 'ADP': t = 'ADP'
                    elif word_obj.upos == 'DET': t = 'DET'
                    else: t = 'UNK'

                    if t != 'UNK':
                        processed_results[t].append(word_obj.text)
        except Exception as e:
            # Hata olsa bile devam et, sadece o batch'i atla
            continue

        # KAYDETME (Her 20 batch - 10k kelime)
        save_counter += 1
        if save_counter % 20 == 0:
            with open(TEMP_SAVE_FILE, 'w', encoding='utf-8') as f:
                json.dump(processed_results, f, ensure_ascii=False)

    # 6. FİNAL
    print("\n   💾 Bitti! Kaydediliyor...")
    final_data = data.copy()
    final_data['OTHER'] = []

    for tag, new_words in processed_results.items():
        if tag in final_data:
            final_data[tag].extend(new_words)
            final_data[tag] = sorted(list(set(final_data[tag])))
        else:
            final_data[tag] = sorted(list(set(new_words)))

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False)

    if os.path.exists(TEMP_SAVE_FILE):
        os.remove(TEMP_SAVE_FILE)

    print(f"🏆 FİNAL DOSYA HAZIR: {OUTPUT_FILE}")

turbo_stanza_safe()

✅ GPU Algılandı, Turbo Mod Başlatılıyor...
   📦 Toplam 'OTHER' Kelime Sayısı: 1129886
   📂 Kayıtlı ilerleme bulundu! Devam ediliyor...
   ⏩ 39686 kelime atlandı. Kalan: 1091019
   🔥 İşlem Başlıyor...


Turbo GPU: 100%|██████████| 2183/2183 [56:42<00:00,  1.56s/it]



   💾 Bitti! Kaydediliyor...
🏆 FİNAL DOSYA HAZIR: /content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json


In [5]:
import json
import random
import os

FILE_PATH = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json'

def check_lexicon_health():
    if not os.path.exists(FILE_PATH):
        print("❌ Dosya bulunamadı! Yolu kontrol et.")
        return

    print("📂 Dosya yükleniyor (Biraz sürebilir)...")
    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"✅ Dosya başarıyla yüklendi!")
    print("="*40)

    total_words = 0

    # Her kategoriden rastgele 5 örnek göster
    for category, word_list in data.items():
        count = len(word_list)
        total_words += count
        print(f"🔹 {category:<10} : {count:,} kelime")

        # Örnekler
        if count > 0:
            sample = random.sample(word_list, min(5, count))
            print(f"   Örnekler: {', '.join(sample)}")
        print("-" * 20)

    print("="*40)
    print(f"📊 TOPLAM KELİME SAYISI: {total_words:,}")

check_lexicon_health()

📂 Dosya yükleniyor (Biraz sürebilir)...
✅ Dosya başarıyla yüklendi!
🔹 OTHER      : 0 kelime
--------------------
🔹 PRON       : 685 kelime
   Örnekler: benzeri, sizgen, şunla, birinede, kendilerinin
--------------------
🔹 NOUN       : 986,200 kelime
   Örnekler: relerin, lappe, canmore, gaviiformes, stanbulkarta
--------------------
🔹 ADJ        : 48,022 kelime
   Örnekler: klimatolojik, mücellâ, gotın, sevmeyenin, edepli
--------------------
🔹 NUM        : 67,259 kelime
   Örnekler: stivers, klavo, felâketzede, botanic, dakara
--------------------
🔹 VERB       : 207,486 kelime
   Örnekler: getirileceksin, başlamayan, dengeleneceğine, yükseltilebilen, uzayacaktı
--------------------
🔹 CONJ       : 253 kelime
   Örnekler: ille velâkin, kaldı ki, görüro, muuttaa, emde
--------------------
🔹 ADV        : 5,315 kelime
   Örnekler: taptaze, kız kıza, memnun, bedava, ciddiyken
--------------------
🔹 ADP        : 3,947 kelime
   Örnekler: kulluğumuz, musavve, milleluci, ağıdıyken, linde
-----

In [6]:
#GEREKSİZ KELİME TEMİZLİĞİ 1
import json
import re
import os
from collections import Counter

# Dosya Yolları
INPUT_FILE = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_LEXICON.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json'

def refinery_logic():
    print("🧹 Temizlik Operasyonu Başlıyor...")

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    clean_lexicon = {
        'NOUN': set(), 'VERB': set(), 'ADJ': set(),
        'ADV': set(), 'PRON': set(), 'CONJ': set(),
        'ADP': set(), 'DET': set(), 'NUM': set(), 'PROPN': set()
    }

    # İstemediğimiz karakterler
    garbage_pattern = re.compile(r'[wqx0-9_@\.\-\'\+]')

    # Fiil temizleyici (Basit Kural Tabanlı Stemmer)
    # Amacımız "getirileceksin" -> "getir" yapmak
    def clean_verb(word):
        # Sondan eklemeli dillerde tersten budama
        suffixes = [
            'mek', 'mak', 'me', 'ma', 'iyor', 'ıyor', 'uyor', 'üyor',
            'ecek', 'acak', 'mış', 'miş', 'muş', 'müş', 'dı', 'di', 'du', 'dü',
            'sın', 'sin', 'sun', 'sün', 'lar', 'ler', 'nız', 'niz'
        ]

        root = word
        # Basit bir döngüyle sondaki ekleri at (Çok agresif olmadan)
        # Kök en az 3 harf kalmalı (yememek -> ye)
        original = word
        for _ in range(3): # 3 katman ek silebilir
            for suf in suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2:
                    root = root[:-len(suf)]
                    break
        return root

    stats = {'deleted': 0, 'kept': 0, 'trimmed': 0}

    for category, words in data.items():
        print(f"   Süzülüyor: {category} ({len(words)} aday)...")

        for word in words:
            word = word.lower().strip()

            # 1. Kural: Çöp karakterler ve Uzunluk
            if garbage_pattern.search(word) or len(word) < 2 or len(word) > 12:
                stats['deleted'] += 1
                continue

            # 2. Kural: Yapışık Kelimeler (Parser Hataları)
            # "yapılmıştırbu" -> sonu 'bu' ile bitiyorsa ve uzunsa şüphelidir
            if (word.endswith('bu') or word.endswith('ve')) and len(word) > 8:
                stats['deleted'] += 1
                continue

            # 3. Kural: Kategoriye Özel Temizlik
            if category == 'VERB':
                root = clean_verb(word)
                if root != word:
                    stats['trimmed'] += 1
                clean_lexicon['VERB'].add(root)

            elif category == 'NOUN':
                # İsimlerde çok fazla çekim eki temizlemiyoruz, FST halletsin
                # Ama çok uzunları almayalım
                if len(word) < 10:
                    clean_lexicon['NOUN'].add(word)

            elif category == 'NUM':
                # Sayı olmayanları at (stivers vs.)
                # Yazıyla yazılan sayılar kalsın (bir, iki, yüz...)
                valid_nums = ["bir", "iki", "üç", "dört", "beş", "altı", "yedi", "sekiz", "dokuz", "on", "yirmi", "otuz", "kırk", "elli", "altmış", "yetmiş", "seksen", "doksan", "yüz", "bin", "milyon", "milyar", "sıfır"]
                if word in valid_nums:
                    clean_lexicon['NUM'].add(word)

            elif category == 'DET':
                # 'yayınlanmıştırbu' gibi hataları elemek için sadece bilinenleri alabiliriz
                # veya kısa olanları tutabiliriz
                if len(word) < 8:
                    clean_lexicon['DET'].add(word)

            else:
                clean_lexicon[category].add(word)

    # Listeye çevirip kaydet
    final_output = {k: sorted(list(v)) for k, v in clean_lexicon.items()}

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, ensure_ascii=False)

    print("\n✅ TEMİZLİK BİTTİ!")
    print(f"   🗑️ Silinen (Çöp/Hata): {stats['deleted']:,}")
    print(f"   ✂️ Budanan (Fiil Kökü): {stats['trimmed']:,}")

    total_clean = sum(len(v) for v in final_output.values())
    print(f"   💎 Kalan Saf Kelime: {total_clean:,}")
    print(f"   📂 Kayıt Yeri: {OUTPUT_FILE}")

    # VERB Örnekleri
    print(f"\n   Fiil Örnekleri (Kök Haline Dönmüş mü?):")
    print(f"   {final_output['VERB'][:10]}")

refinery_logic()

🧹 Temizlik Operasyonu Başlıyor...
   Süzülüyor: OTHER (0 aday)...
   Süzülüyor: PRON (685 aday)...
   Süzülüyor: NOUN (986200 aday)...
   Süzülüyor: ADJ (48022 aday)...
   Süzülüyor: NUM (67259 aday)...
   Süzülüyor: VERB (207486 aday)...
   Süzülüyor: CONJ (253 aday)...
   Süzülüyor: ADV (5315 aday)...
   Süzülüyor: ADP (3947 aday)...
   Süzülüyor: DET (505 aday)...

✅ TEMİZLİK BİTTİ!
   🗑️ Silinen (Çöp/Hata): 305,336
   ✂️ Budanan (Fiil Kökü): 42,839
   💎 Kalan Saf Kelime: 650,514
   📂 Kayıt Yeri: /content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json

   Fiil Örnekleri (Kök Haline Dönmüş mü?):
   ['aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaa', 'aaaaaad', 'aaadır', 'aban', 'abanabil', 'abanabilir', 'abanacağım']


In [7]:
#GEREKSİZ EKLER TEMİZLİĞİ
import json
import re
import os

INPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json'

def acid_bath_cleaning():
    print("🧪 Asit Banyosu Başlıyor (Agresif Temizlik)...")

    if not os.path.exists(INPUT_FILE):
        print("❌ Önceki dosya yok! Lütfen önce Rafineri 1'i çalıştır.")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    final_lexicon = {k: set() for k in data.keys()}

    # Türkçe Sesli Harfler
    vowels = set("aeıioöuü")
    # 3 tane yan yana aynı harf yakalayan regex (aaa, bbb)
    triple_char_regex = re.compile(r'(.)\1\1')

    # Agresif Fiil Budayıcı
    def aggressive_verb_stemmer(word):
        # Önce çekim ekleri (Tense/Person)
        inflectional_suffixes = [
            'casına', 'cesine', 'mı', 'mi', 'mu', 'mü',
            'tır', 'tir', 'tur', 'tür', 'dır', 'dir', 'dur', 'dür',
            'nız', 'niz', 'nuz', 'nüz', 'lar', 'ler', 'sın', 'sin', 'sun', 'sün',
            'ız', 'iz', 'uz', 'üz', 'ım', 'im', 'um', 'üm',
            'dı', 'di', 'du', 'dü', 'tı', 'ti', 'tu', 'tü',
            'mış', 'miş', 'muş', 'müş', 'yacak', 'yecek', 'acak', 'ecek',
            'yor', 'ıyor', 'iyor', 'uyor', 'üyor',
            'malı', 'meli', 'sa', 'se', 'a', 'e'
        ]

        # Sonra yapım/çatı ekleri (Derivational) - Sıra önemli!
        derivational_suffixes = [
            'abil', 'ebil', 'yabil', 'yebil', # Yeterlilik
            'iver', 'ıver', 'uver', 'üver',   # Tezlik
            'yaz', 'eyaz',                    # Yaklaşma
            'dur', 'edur',                    # Süerlik
            'mek', 'mak', 'me', 'ma'          # Mastar
        ]

        root = word

        # 1. Tur: Çekim eklerini at
        changed = True
        while changed:
            changed = False
            for suf in inflectional_suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2: # Kök en az 2 harf kalsın (ye, de)
                    root = root[:-len(suf)]
                    changed = True
                    break

        # 2. Tur: Çatı/Birleşik eklerini at (aban-abil -> aban)
        changed = True
        while changed:
            changed = False
            for suf in derivational_suffixes:
                if root.endswith(suf) and len(root) > len(suf) + 2:
                    root = root[:-len(suf)]
                    changed = True
                    break

        return root

    stats = {'deleted': 0, 'trimmed': 0}

    for category, word_list in data.items():
        print(f"   İşleniyor: {category} ({len(word_list)} kelime)...")

        for word in word_list:
            original_word = word

            # KURAL 1: 'aaaa' kontrolü
            if triple_char_regex.search(word):
                stats['deleted'] += 1
                continue

            # KURAL 2: Sesli harf kontrolü (Sessiz harflerden oluşan kısaltmaları at)
            if not any(char in vowels for char in word):
                stats['deleted'] += 1
                continue

            # KURAL 3: Çok uzun ve anlamsız kelimeler
            if len(word) > 15: # Türkçe kökler bu kadar uzun olmaz
                stats['deleted'] += 1
                continue

            # KURAL 4: Agresif Stemming
            if category == 'VERB':
                root = aggressive_verb_stemmer(word)
                if root != word:
                    stats['trimmed'] += 1
                final_lexicon['VERB'].add(root)
            elif category == 'NOUN':
                 # İsimlerde de basit çoğul eki temizliği yapalım
                 if word.endswith('lar') and len(word) > 5:
                     final_lexicon['NOUN'].add(word[:-3])
                     stats['trimmed'] += 1
                 elif word.endswith('ler') and len(word) > 5:
                     final_lexicon['NOUN'].add(word[:-3])
                     stats['trimmed'] += 1
                 else:
                     final_lexicon['NOUN'].add(word)
            else:
                final_lexicon[category].add(word)

    # Dosyayı kaydet
    output_data = {k: sorted(list(v)) for k, v in final_lexicon.items()}

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False)

    total_clean = sum(len(v) for v in output_data.values())

    print("\n🧪 ASİT BANYOSU TAMAMLANDI!")
    print(f"   🗑️ Eritilen (aaaa/çöp): {stats['deleted']:,}")
    print(f"   ✂️ Kesilen (aban-abil): {stats['trimmed']:,}")
    print(f"   💎 Kalan Saf Kök: {total_clean:,}")
    print(f"   📂 Yeni Dosya: {OUTPUT_FILE}")

    print(f"\n   Yeni Fiil Örnekleri (İlk 10):")
    print(f"   {output_data['VERB'][:10]}")

acid_bath_cleaning()

🧪 Asit Banyosu Başlıyor (Agresif Temizlik)...
   İşleniyor: NOUN (507171 kelime)...
   İşleniyor: VERB (93679 kelime)...
   İşleniyor: ADJ (40859 kelime)...
   İşleniyor: ADV (4507 kelime)...
   İşleniyor: PRON (571 kelime)...
   İşleniyor: CONJ (196 kelime)...
   İşleniyor: ADP (3471 kelime)...
   İşleniyor: DET (42 kelime)...
   İşleniyor: NUM (18 kelime)...
   İşleniyor: PROPN (0 kelime)...

🧪 ASİT BANYOSU TAMAMLANDI!
   🗑️ Eritilen (aaaa/çöp): 6,010
   ✂️ Kesilen (aban-abil): 52,463
   💎 Kalan Saf Kök: 613,755
   📂 Yeni Dosya: /content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json

   Yeni Fiil Örnekleri (İlk 10):
   ['aban', 'abanabilir', 'abanacağ', 'aband', 'abandık', 'abandın', 'abandırır', 'abanmışsı', 'abanozlaş', 'abanozlaşır']


In [9]:
import json
import os

INPUT_FILE = '/content/drive/MyDrive/FSTurk/CLEAN_ROOTS_V2.json'
OUTPUT_FILE = '/content/drive/MyDrive/FSTurk/MASTER_FINAL_ROOTS.json'

def turbo_smart_reduction():
    print("🚀 TURBO İndirgeme Başlıyor (Set Lookup Strategy)...")

    if not os.path.exists(INPUT_FILE):
        print("❌ Dosya bulunamadı!")
        return

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Ek listesi (Aynı liste)
    valid_suffixes = tuple([ # Tuple yaptık ki startswith daha hızlı çalışsın
        'mak', 'mek', 'ma', 'me', 'ış', 'iş', 'uş', 'üş',
        'yor', 'yabil', 'yebil', 'yacak', 'yecek', 'r', 'ar', 'er', 'ır', 'ir', 'ur', 'ür',
        'dı', 'di', 'du', 'dü', 'tı', 'ti', 'tu', 'tü', 'mış', 'miş', 'muş', 'müş',
        'sın', 'sin', 'sun', 'sün', 'nız', 'niz', 'k', 'n', 'm',
        'abil', 'ebil', 'iver', 'ıver', 'dur', 'calama', 'celeme',
        'lar', 'ler', 'nın', 'nin', 'nun', 'nün', 'dan', 'den', 'tan', 'ten',
        'da', 'de', 'ta', 'te', 'ya', 'ye', 'yı', 'yi', 'yu', 'yü',
        'lı', 'li', 'lu', 'lü', 'sız', 'siz', 'suz', 'süz', 'lık', 'lik', 'luk', 'lük',
        'cı', 'ci', 'cu', 'cü', 'ça', 'çe', 'ca', 'ce', 'ki', 'kü', 'sı', 'si',
        'ı', 'i', 'u', 'ü' # Belirtme hali eklerini de ekledim
    ])

    final_lexicon = {}
    stats = {'deleted': 0, 'kept': 0}

    for category, word_list in data.items():
        print(f"   ⚡ Hızlandırılmış Tarama: {category} ({len(word_list)} kelime)...")

        # 1. Kelimeleri yine kısadan uzuna sırala
        sorted_words = sorted(list(word_list), key=len)

        # SET kullanıyoruz (Arama hızı O(1))
        kept_roots = set()

        category_deleted = 0

        for word in sorted_words:
            # Çok kısa kelimeleri direkt al
            if len(word) <= 3:
                kept_roots.add(word)
                continue

            is_derivative = False

            # YENİ MANTIK: Kelimenin kendisi içinde "prefix" arıyoruz
            # word = "kitaplık" -> i=3'ten başla
            # kitaplık -> 'kit', 'kita', 'kitap'...

            # Kelimenin uzunluğu kadar döngü (Max 15-20 tur, 1 Milyon tur değil!)
            for i in range(3, len(word)):
                prefix = word[:i] # Potansiyel kök
                suffix = word[i:] # Geriye kalan

                # 1. Bu prefix bizim elimizdeki köklerde var mı?
                if prefix in kept_roots:
                    # 2. Geriye kalan kısım geçerli bir ek mi?
                    if suffix.startswith(valid_suffixes):
                        is_derivative = True
                        break

            if is_derivative:
                category_deleted += 1
                stats['deleted'] += 1
            else:
                kept_roots.add(word)

        final_lexicon[category] = sorted(list(kept_roots))
        print(f"      -> {category} bitti. {category_deleted} kelime elendi.")

    # Dosyayı kaydet
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(final_lexicon, f, ensure_ascii=False)

    total = sum(len(v) for v in final_lexicon.values())

    print("\n🏆 TURBO OPERASYON TAMAMLANDI!")
    print(f"   🗑️ Toplam Elenen: {stats['deleted']:,}")
    print(f"   💎 Kalan Saf Kökler: {total:,}")
    print(f"   📂 Dosya: {OUTPUT_FILE}")

    # Kontrol
    if 'VERB' in final_lexicon:
        sample = [w for w in final_lexicon['VERB'] if w.startswith('aban')]
        print(f"\n   'aban' Kontrolü: {sample}")

turbo_smart_reduction()

🚀 TURBO İndirgeme Başlıyor (Set Lookup Strategy)...
   ⚡ Hızlandırılmış Tarama: NOUN (492050 kelime)...
      -> NOUN bitti. 348169 kelime elendi.
   ⚡ Hızlandırılmış Tarama: VERB (72152 kelime)...
      -> VERB bitti. 54776 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADJ (40782 kelime)...
      -> ADJ bitti. 12758 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADV (4502 kelime)...
      -> ADV bitti. 608 kelime elendi.
   ⚡ Hızlandırılmış Tarama: PRON (571 kelime)...
      -> PRON bitti. 385 kelime elendi.
   ⚡ Hızlandırılmış Tarama: CONJ (193 kelime)...
      -> CONJ bitti. 8 kelime elendi.
   ⚡ Hızlandırılmış Tarama: ADP (3445 kelime)...
      -> ADP bitti. 619 kelime elendi.
   ⚡ Hızlandırılmış Tarama: DET (42 kelime)...
      -> DET bitti. 4 kelime elendi.
   ⚡ Hızlandırılmış Tarama: NUM (18 kelime)...
      -> NUM bitti. 0 kelime elendi.
   ⚡ Hızlandırılmış Tarama: PROPN (0 kelime)...
      -> PROPN bitti. 0 kelime elendi.

🏆 TURBO OPERASYON TAMAMLANDI!
   🗑️ Toplam Elenen: 417,327
  

# 🔄 Universal Dependencies (UD) → FST Eşleme Modülü

Bu bölüm, UD Turkish-IMST formatındaki anotasyonları FST formatına dönüştürür.

**FST formatı:** `lemma+POS+FEATURE1+FEATURE2+...`

**UD formatı:** FORM | LEMMA | UPOS | FEATS (Key=Value|...)


In [ ]:
# ============================================================
# BÖLÜM 1: UD → FST EŞLEMİ TAM TABLOLARl
# ============================================================

# ── 1A. UPOS → FST POS Eşleme ──
UPOS_TO_FST_POS = {
    'NOUN':  'NOUN',
    'PROPN': 'PROPN',
    'VERB':  'VERB',
    'ADJ':   'ADJ',
    'ADV':   'ADV',
    'PRON':  'PRON',
    'DET':   'DET',       # FST'de ayrı kategori yok, PRON/ADJ'ye düşebilir
    'ADP':   'POSTP',     # Türkçe'de postposition
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ',
    'NUM':   'NUM',
    'AUX':   'AUX',       # FST'de genelde copula (+COP.*) olarak modellenir
    'INTJ':  'INTERJ',
    'PUNCT': 'PUNCT',
    'X':     'UNK',
    'SYM':   'PUNCT',
}

# ── 1B. Case (Durum Eki) Eşleme ──
CASE_MAP = {
    'Nom': '',           # İşaretsiz (unmarked) - FST'de tag yok
    'Acc': '+ACC',
    'Dat': '+DAT',
    'Loc': '+LOC',
    'Abl': '+ABL',
    'Gen': '+GEN',
    'Ins': '+INS',
    'Equ': '+EQU',       # -ca/-ce eşitlik
}

# ── 1C. Number (Sayı) Eşleme ──
NUMBER_MAP = {
    'Sing': '',          # Tekil - işaretsiz
    'Plur': '+PL',
}

# ── 1D. Person (Kişi) Eşleme ──
# Number + Person birleştirilerek FST person tag üretilir
PERSON_NUMBER_MAP = {
    ('Sing', '1'): '+1SG',
    ('Sing', '2'): '+2SG',
    ('Sing', '3'): '+3SG',   # Bazı bağlamlarda boş olabilir
    ('Plur', '1'): '+1PL',
    ('Plur', '2'): '+2PL',
    ('Plur', '3'): '+3PL',
}

# ── 1E. Tense (Zaman) Eşleme ──
TENSE_MAP = {
    'Past': '+PAST',
    'Pres': '',          # Şimdiki zaman - Aspect ile birlikte değerlendirilir
    'Fut':  '+FUT',
}

# ── 1F. Aspect (Görünüş) Eşleme ──
ASPECT_MAP = {
    'Perf': '',          # Perfective - genelde tense ile birlikte
    'Prog': '+PRES.CONT',  # -Iyor (şimdiki zaman sürerlik)
    'Hab':  '+AOR',      # Geniş zaman
    'Rapid':'+IVER',     # Tezlik fiili (nadiren)
}

# ── 1G. Mood (Kip) Eşleme ──
MOOD_MAP = {
    'Ind':  '',          # Bildirme kipi - varsayılan
    'Imp':  '+IMP',
    'Opt':  '+OPT',      # İstek kipi (-a/-e)
    'Des':  '+OPT',      # Dilek kipi (UD'de Desiderative = FST'de OPT)
    'Nec':  '+NEC',      # Gereklilik (-malı/-meli)
    'Pot':  '+ABIL',     # Yeterlilik (-abil/-ebil)
    'Cnd':  '+COND',     # Şart kipi (-sa/-se)
    'Gen':  '+COP.PRES', # Genelleyici -(I)dIr
    'CndGen':'+COND',    # Koşullu genelleyici (nadir)
}

# ── 1H. Voice (Çatı) Eşleme ──
VOICE_MAP = {
    'Pass': '+PASS',     # Edilgen
    'Cau':  '+CAUS',     # Ettirgen
    'Rfl':  '+REFL',     # Dönüşlü
    'Rcp':  '+RECIP',    # İşteş
    'CauPass':'+CAUS+PASS', # Ettirgen + Edilgen
}

# ── 1I. Polarity (Olumsuzluk) Eşleme ──
POLARITY_MAP = {
    'Pos': '',           # Olumlu - işaretsiz
    'Neg': '+NEG',
}

# ── 1J. Possessive (İyelik) Eşleme ──
# UD: Number[psor] + Person[psor] → FST: +POSS.XY
POSS_MAP = {
    ('Sing', '1'): '+POSS.1SG',
    ('Sing', '2'): '+POSS.2SG',
    ('Sing', '3'): '+POSS.3SG',
    ('Plur', '1'): '+POSS.1PL',
    ('Plur', '2'): '+POSS.2PL',
    ('Plur', '3'): '+POSS.3PL',
}

# ── 1K. Evident (Kanıtsallık) Eşleme ──
EVIDENT_MAP = {
    'Fh':  '',           # Birinci elden (firsthand) - varsayılan
    'Nfh': '+INFER',     # Duyulan geçmiş (-mIş)
}

# ── 1L. VerbForm (Fiil Biçimi) Eşleme ──
VERBFORM_MAP = {
    'Part':  '',         # Sıfat-fiil - bağlama göre analiz
    'Conv':  '+CVB',     # Zarf-fiil (belirli alt tip bağlama göre)
    'Vnoun': '',         # İsim-fiil (-mA, -mAk, -Iş)
    'Fin':   '',         # Çekimli fiil - varsayılan
}

# ── 1M. Derivational (Türetim) Eşleme ──
# UD'de doğrudan yok, ancak bazı NAdj etiketleri ipucu verir
# FST: +DER.lık, +DER.cı, +DER.sız, +DER.lı vb.
# Bu özellikler UD'den otomatik çıkarılamaz, lemma+XPOS'tan tahmin edilebilir

print('✅ Eşleme tabloları yüklendi.')
print(f'   UPOS eşleme: {len(UPOS_TO_FST_POS)} kategori')
print(f'   Case eşleme: {len(CASE_MAP)} durum')
print(f'   Tense eşleme: {len(TENSE_MAP)} zaman')
print(f'   Mood eşleme: {len(MOOD_MAP)} kip')
print(f'   Voice eşleme: {len(VOICE_MAP)} çatı')


## ⚠️ Belirsiz / Doğrudan Eşlenemeyen Özellikler

| Durum | UD Tarafı | FST Tarafı | Karar |
|-------|-----------|------------|-------|
| **AUX (yardımcı fiil)** | `AUX` (değil, i-) | `+COP.*` (copula eki) | AUX → COP altı etiketlerine dönüştürülür |
| **DET (belirteç)** | `DET` (bir, bu, o) | FST'de yok | DET kalır veya PRON'a atanır |
| **Aspect=Perf** | Bitmişlik | FST'de yok | Tense ile birlikte yorumlanır |
| **Polite=Infm** | Kayıt dışı | FST'de yok | İhmal edilir (bilgi kaybı) |
| **PronType** | Dem/Prs/Qes... | FST'de yok | İhmal, ama geri çevrilebilirlik kaybı |
| **3SG varsayılan** | Açık `Person=3` | FST bazen boş bırakır | Tutarlılık için +3SG yazılır |
| **NAdj (XPOS)** | ADJ/NOUN arası | NOUN veya ADJ | UPOS'a güvenilir |
| **Türetim ekleri** | UD'de `li/sız/lık/cı` | `+DER.*` | Lemma farkından çıkarım yapılır |
| **VerbForm=Part** | Sıfat-fiil | FST'de explicit yok | VERB+Part notasyonu eklenir |
| **VerbForm=Conv** | Zarf-fiil | `+CVB.*` | Alt tip belirsiz, genel +CVB |

### 🔴 FST Daha Zengin Olduğu Durumlar
- **+EMPH** (vurgulama): UD'de karşılığı yok → UD→FST'de üretilemez
- **+KI** (relativizer): UD'de `ADP` `ki` olarak ayrı token → birleştirme gerekir
- **+CVB.INCA/KEN/IP/MADAN**: UD sadece `Conv` der, FST alt tip belirtir
- **+DER.lık/cı/sız/lı**: UD'de morfolojik türetim belirtilmez

### 🟡 UD Daha Zengin Olduğu Durumlar
- **Polite=Infm/Form**: FST'de karşılık yok → ihmal
- **PronType=Prs/Dem/Qes**: FST zamirleri alt tiplere ayırmaz → ihmal
- **NumType=Card/Ord**: FST'de NUM tek kategori → ihmal


In [ ]:
# ============================================================
# BÖLÜM 2: map_ud_to_fst() ANA FONKSİYONU
# ============================================================

def parse_ud_feats(feats_str):
    """UD FEATS string'ini dict'e çevirir: 'Case=Acc|Number=Sing' → {'Case':'Acc', ...}"""
    if not feats_str or feats_str == '_':
        return {}
    result = {}
    for pair in feats_str.split('|'):
        if '=' in pair:
            k, v = pair.split('=', 1)
            result[k] = v
    return result


def map_ud_to_fst(lemma, upos, feats_dict, xpos=None):
    """
    UD anotasyonunu FST analiz string'ine dönüştürür.
    
    Args:
        lemma (str): UD lemma
        upos (str): UD UPOS etiketi
        feats_dict (dict): UD özellik sözlüğü {'Case':'Acc', ...}
        xpos (str): İsteğe bağlı XPOS etiketi (NAdj algılama için)
    
    Returns:
        str: FST formatında analiz string'i
        list: Uyarılar/bilgi kayıpları listesi
    """
    warnings = []
    tags = []
    
    # ── 1. POS Eşleme ──
    fst_pos = UPOS_TO_FST_POS.get(upos, 'UNK')
    if fst_pos == 'UNK':
        warnings.append(f'Bilinmeyen UPOS: {upos}')
    
    # AUX özel işlem: copula olarak eşle
    if upos == 'AUX':
        return _map_aux_to_fst(lemma, feats_dict, warnings)
    
    # PUNCT özel işlem
    if upos == 'PUNCT':
        punct_map = {'.': '+PUNCT.period', ',': '+PUNCT.comma', 
                     '?': '+PUNCT.question', '!': '+PUNCT.exclamation',
                     ':': '+PUNCT.colon', ';': '+PUNCT.semicolon',
                     '...': '+PUNCT.ellipsis', '-': '+PUNCT.dash'}
        suffix = punct_map.get(lemma, '+PUNCT')
        return f'{lemma}{suffix}', warnings
    
    # ── 2. Fiil Morfolojisi ──
    if fst_pos == 'VERB':
        return _map_verb_to_fst(lemma, feats_dict, warnings)
    
    # ── 3. İsim/Sıfat/Zamir Morfolojisi ──
    if fst_pos in ('NOUN', 'PROPN', 'ADJ', 'PRON', 'NUM', 'DET'):
        return _map_nominal_to_fst(lemma, fst_pos, feats_dict, warnings)
    
    # ── 4. Basit Kategoriler (ADV, CONJ, POSTP, INTERJ) ──
    return f'{lemma}+{fst_pos}', warnings


def _map_verb_to_fst(lemma, feats, warnings):
    """Fiil morfolojisini FST formatına dönüştürür."""
    parts = [f'{lemma}+VERB']
    
    # Çatı (Voice)
    voice = feats.get('Voice', '')
    if voice and voice in VOICE_MAP:
        parts.append(VOICE_MAP[voice])
    
    # Yeterlilik (Mood=Pot → +ABIL)
    mood = feats.get('Mood', 'Ind')
    if mood == 'Pot':
        parts.append('+ABIL')
    
    # Olumsuzluk (Polarity)
    polarity = feats.get('Polarity', 'Pos')
    neg_tag = POLARITY_MAP.get(polarity, '')
    if neg_tag:
        parts.append(neg_tag)
    
    # VerbForm kontrol (Part, Conv, Vnoun)
    verb_form = feats.get('VerbForm', '')
    
    if verb_form == 'Conv':
        # Zarf-fiil - UD alt tip belirtmez, genel CVB
        parts.append('+CVB')
        warnings.append('CVB alt tipi (INCA/KEN/IP/MADAN) UD\'den çıkarılamaz')
        return ''.join(parts), warnings
    
    if verb_form == 'Part':
        # Sıfat-fiil
        parts.append('+PART')
    
    if verb_form == 'Vnoun':
        # İsim-fiil
        parts.append('+VNOUN')
    
    # Zaman + Görünüş (Tense + Aspect)
    tense = feats.get('Tense', '')
    aspect = feats.get('Aspect', '')
    evident = feats.get('Evident', '')
    
    # Duyulan geçmiş (-mIş)
    if evident == 'Nfh':
        parts.append('+INFER')
    elif aspect == 'Prog':
        parts.append('+PRES.CONT')
    elif aspect == 'Hab':
        parts.append('+AOR')
    elif tense == 'Past':
        parts.append('+PAST')
    elif tense == 'Fut':
        parts.append('+FUT')
    elif tense == 'Pres' and aspect == 'Perf':
        pass  # Geniş zaman / şimdiki bağlamda zaman eki yok
    
    # Kip (Mood) - Pot zaten yukarıda işlendi
    if mood and mood not in ('Ind', 'Pot'):
        mood_tag = MOOD_MAP.get(mood, '')
        if mood_tag:
            parts.append(mood_tag)
    
    # Kişi + Sayı
    number = feats.get('Number', '')
    person = feats.get('Person', '')
    if number and person:
        pn_tag = PERSON_NUMBER_MAP.get((number, person), '')
        if pn_tag:
            parts.append(pn_tag)
    
    # İyelik (possessor - fiil sıfat-fiillerinde)
    npsor = feats.get('Number[psor]', '')
    ppsor = feats.get('Person[psor]', '')
    if npsor and ppsor:
        poss_tag = POSS_MAP.get((npsor, ppsor), '')
        if poss_tag:
            parts.append(poss_tag)
    
    # Durum (fiil-isimleşmelerde case olabilir)
    case = feats.get('Case', '')
    if case and case != 'Nom':
        case_tag = CASE_MAP.get(case, '')
        if case_tag:
            parts.append(case_tag)
    
    # Polite bilgi kaybı
    if 'Polite' in feats:
        warnings.append(f'Polite={feats["Polite"]} FST\'de karşılığı yok')
    
    return ''.join(parts), warnings


def _map_nominal_to_fst(lemma, fst_pos, feats, warnings):
    """İsim/Sıfat/Zamir morfolojisini FST formatına dönüştürür."""
    parts = [f'{lemma}+{fst_pos}']
    
    # Çoğul
    number = feats.get('Number', 'Sing')
    if number == 'Plur':
        parts.append('+PL')
    
    # İyelik (Possessive)
    npsor = feats.get('Number[psor]', '')
    ppsor = feats.get('Person[psor]', '')
    if npsor and ppsor:
        poss_tag = POSS_MAP.get((npsor, ppsor), '')
        if poss_tag:
            parts.append(poss_tag)
    
    # Durum (Case)
    case = feats.get('Case', '')
    if case:
        case_tag = CASE_MAP.get(case, '')
        if case_tag:
            parts.append(case_tag)
    
    # PronType bilgi kaybı
    if 'PronType' in feats:
        warnings.append(f'PronType={feats["PronType"]} FST\'de yok')
    
    return ''.join(parts), warnings


def _map_aux_to_fst(lemma, feats, warnings):
    """AUX (yardımcı fiil/copula) eşlemesi."""
    # Türkçe'de AUX genelde copula "i-" veya "değil"
    parts = [f'{lemma}+AUX']
    
    tense = feats.get('Tense', '')
    mood = feats.get('Mood', '')
    evident = feats.get('Evident', '')
    polarity = feats.get('Polarity', '')
    
    if polarity == 'Neg':
        parts.append('+NEG')
    
    if evident == 'Nfh':
        parts.append('+COP.EVID')
    elif tense == 'Past':
        parts.append('+COP.PAST')
    elif mood == 'Gen':
        parts.append('+COP.PRES')
    elif mood == 'Cnd':
        parts.append('+COP.COND')
    else:
        parts.append('+COP')
    
    # Kişi
    number = feats.get('Number', '')
    person = feats.get('Person', '')
    if number and person:
        pn_tag = PERSON_NUMBER_MAP.get((number, person), '')
        if pn_tag:
            parts.append(pn_tag)
    
    return ''.join(parts), warnings


print('✅ map_ud_to_fst() fonksiyonu hazır.')


## 🧪 Simülasyon: 10 UD Token → FST Çıktısı

Aşağıda UD Turkish-IMST'den seçilmiş 10 token üzerinde dönüşüm simülasyonu yapılmaktadır.


In [ ]:
# ============================================================
# BÖLÜM 3: 10 ÖRNEK UD TOKEN SİMÜLASYONU
# ============================================================

import pandas as pd

# UD Turkish-IMST'den gerçek örnekler
test_tokens = [
    # (FORM, LEMMA, UPOS, FEATS_str)
    ('seviyorum', 'sev', 'VERB',
     'Aspect=Prog|Mood=Ind|Number=Sing|Person=1|Polarity=Pos|Polite=Infm|Tense=Pres'),
    ('yaşlanmayacağız', 'yaşlan', 'VERB',
     'Aspect=Perf|Mood=Ind|Number=Plur|Person=1|Polarity=Neg|Tense=Fut'),
    ('Seni', 'sen', 'PRON',
     'Case=Acc|Number=Sing|Person=2|PronType=Prs'),
    ('limandan', 'liman', 'NOUN',
     'Case=Abl|Number=Sing|Person=3'),
    ('Aşkımız', 'aşk', 'NOUN',
     'Case=Nom|Number=Sing|Number[psor]=Plur|Person=3|Person[psor]=1'),
    ('eskimeyecek', 'eski', 'VERB',
     'Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Polarity=Neg|Tense=Fut'),
    ('demiş', 'de', 'VERB',
     'Aspect=Perf|Evident=Nfh|Mood=Ind|Number=Sing|Person=3|Polarity=Pos|Tense=Past'),
    ('öğrencisiyle', 'öğrenci', 'NOUN',
     'Case=Ins|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3'),
    ('çıkınca', 'çık', 'VERB',
     'Aspect=Perf|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Conv'),
    ('bakın', 'bak', 'VERB',
     'Aspect=Perf|Mood=Imp|Number=Plur|Person=2|Polarity=Pos|Tense=Pres'),
]

print('🧪 UD → FST SİMÜLASYON SONUÇLARI')
print('=' * 90)
print(f'{"FORM":<20} {"LEMMA":<12} {"UPOS":<8} {"FST Çıktısı":<35} {"Uyarılar"}')
print('-' * 90)

results = []
for form, lemma, upos, feats_str in test_tokens:
    feats_dict = parse_ud_feats(feats_str)
    fst_output, warns = map_ud_to_fst(lemma, upos, feats_dict)
    warn_str = '; '.join(warns) if warns else '-'
    print(f'{form:<20} {lemma:<12} {upos:<8} {fst_output:<35} {warn_str}')
    results.append({
        'Form': form, 'Lemma': lemma, 'UPOS': upos,
        'FEATS': feats_str, 'FST_Output': fst_output, 'Warnings': warn_str
    })

print('\n✅ Simülasyon tamamlandı.')

# DataFrame olarak da göster
df_sim = pd.DataFrame(results)
df_sim


In [ ]:
# ============================================================
# BÖLÜM 4: CoNLL-U DOSYASINI OKUYUP TOPLU FST DÖNÜŞÜMÜ
# ============================================================

import os

def parse_conllu(filepath):
    """CoNLL-U dosyasını parse eder, token listesi döndürür."""
    tokens = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split('\t')
            if len(parts) < 10:
                continue
            # Multi-word token satırlarını atla (1-2 gibi)
            if '-' in parts[0] or '.' in parts[0]:
                continue
            tokens.append({
                'id': parts[0],
                'form': parts[1],
                'lemma': parts[2],
                'upos': parts[3],
                'xpos': parts[4],
                'feats': parts[5],
            })
    return tokens


def batch_ud_to_fst(tokens):
    """Token listesini toplu olarak FST formatına dönüştürür."""
    results = []
    warn_counts = {}
    
    for tok in tokens:
        feats_dict = parse_ud_feats(tok['feats'])
        fst_out, warns = map_ud_to_fst(
            tok['lemma'], tok['upos'], feats_dict, tok.get('xpos')
        )
        results.append({
            'form': tok['form'],
            'lemma': tok['lemma'],
            'upos': tok['upos'],
            'feats': tok['feats'],
            'fst_analysis': fst_out,
            'warnings': warns,
        })
        for w in warns:
            warn_counts[w] = warn_counts.get(w, 0) + 1
    
    return results, warn_counts


# ── Dosya yolunu ayarla ──
# Yerel ortam için:
CONLLU_PATH = os.path.join('..', '..', 'tr_imst-ud-test.conllu')
# Colab için alternatif:
# CONLLU_PATH = '/content/drive/MyDrive/FSTurk/tr_imst-ud-test.conllu'

if os.path.exists(CONLLU_PATH):
    tokens = parse_conllu(CONLLU_PATH)
    print(f'📦 {len(tokens)} token okundu.')
    
    fst_results, warn_summary = batch_ud_to_fst(tokens)
    
    # İstatistikler
    print(f'✅ {len(fst_results)} token FST formatına dönüştürüldü.')
    print(f'\n⚠️ Uyarı Özeti ({len(warn_summary)} farklı uyarı tipi):')
    for w, count in sorted(warn_summary.items(), key=lambda x:-x[1])[:10]:
        print(f'   {count:>5}x | {w}')
    
    # İlk 20 örneği göster
    print(f'\n📋 İlk 20 Dönüşüm Örneği:')
    print(f'{"FORM":<20} {"UPOS":<8} {"FST Analizi"}')
    print('-' * 60)
    for r in fst_results[:20]:
        print(f'{r["form"]:<20} {r["upos"]:<8} {r["fst_analysis"]}')
else:
    print(f'❌ CoNLL-U dosyası bulunamadı: {CONLLU_PATH}')
    print('   Lütfen dosya yolunu kontrol edin.')


In [ ]:
# ============================================================
# BÖLÜM 5: FST ANALİZLERİNİ DIŞA AKTARMA
# ============================================================

import json

def export_fst_analyses(results, output_path):
    """FST analizlerini JSON formatında kaydeder."""
    export_data = []
    for r in results:
        export_data.append({
            'form': r['form'],
            'lemma': r['lemma'],
            'upos': r['upos'],
            'ud_feats': r['feats'],
            'fst_analysis': r['fst_analysis'],
            'warnings': r['warnings'],
        })
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    print(f'💾 {len(export_data)} analiz kaydedildi: {output_path}')


def export_fst_plain(results, output_path):
    """FST analizlerini düz metin formatında kaydeder (FST test girişi için)."""
    with open(output_path, 'w', encoding='utf-8') as f:
        for r in results:
            f.write(f'{r["form"]}\t{r["fst_analysis"]}\n')
    
    print(f'📝 Düz metin çıktısı: {output_path}')


# Kaydet
if 'fst_results' in dir() and fst_results:
    # JSON formatı
    export_fst_analyses(fst_results, 'ud_to_fst_mapped.json')
    # Düz metin
    export_fst_plain(fst_results, 'ud_to_fst_mapped.txt')
    print('\n✅ Tüm çıktılar hazır!')
else:
    print('⚠️ Önce CoNLL-U dosyasını yükleyin (Bölüm 4).')


In [ ]:
# ============================================================
# BÖLÜM 6: DOĞRULAMA ve İSTATİSTİKLER
# ============================================================

if 'fst_results' in dir() and fst_results:
    from collections import Counter
    
    # POS dağılımı
    pos_counter = Counter(r['upos'] for r in fst_results)
    print('📊 UPOS Dağılımı:')
    for pos, count in pos_counter.most_common():
        fst_eq = UPOS_TO_FST_POS.get(pos, '?')
        print(f'   {pos:<8} → {fst_eq:<8} : {count:>5} token')
    
    # Uyarısız token oranı
    clean = sum(1 for r in fst_results if not r['warnings'])
    total = len(fst_results)
    print(f'\n🎯 Temiz Dönüşüm Oranı: {clean}/{total} ({100*clean/total:.1f}%)')
    
    # Fiil kipi dağılımı
    verb_moods = []
    for r in fst_results:
        if r['upos'] == 'VERB':
            fd = parse_ud_feats(r['feats'])
            m = fd.get('Mood', 'Ind')
            verb_moods.append(m)
    mood_counter = Counter(verb_moods)
    print(f'\n📊 Fiil Kipi Dağılımı:')
    for mood, count in mood_counter.most_common():
        fst_eq = MOOD_MAP.get(mood, '?')
        print(f'   {mood:<10} → {fst_eq:<12} : {count:>5} fiil')
    
    print('\n✅ Doğrulama tamamlandı.')
else:
    print('⚠️ Önce CoNLL-U dosyasını yükleyin (Bölüm 4).')


# 🔄 Universal Dependencies (UD) → FST Eşleme Modülü

Bu bölüm, UD Turkish-IMST formatındaki anotasyonları FST formatına dönüştürür.

**FST formatı:** `lemma+POS+FEATURE1+FEATURE2+...`

**UD formatı:** FORM | LEMMA | UPOS | FEATS (Key=Value|...)


In [ ]:
# ============================================================
# BÖLÜM 1: UD → FST EŞLEMİ TAM TABLOLARl
# ============================================================

# ── 1A. UPOS → FST POS Eşleme ──
UPOS_TO_FST_POS = {
    'NOUN':  'NOUN',
    'PROPN': 'PROPN',
    'VERB':  'VERB',
    'ADJ':   'ADJ',
    'ADV':   'ADV',
    'PRON':  'PRON',
    'DET':   'DET',       # FST'de ayrı kategori yok, PRON/ADJ'ye düşebilir
    'ADP':   'POSTP',     # Türkçe'de postposition
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ',
    'NUM':   'NUM',
    'AUX':   'AUX',       # FST'de genelde copula (+COP.*) olarak modellenir
    'INTJ':  'INTERJ',
    'PUNCT': 'PUNCT',
    'X':     'UNK',
    'SYM':   'PUNCT',
}

# ── 1B. Case (Durum Eki) Eşleme ──
CASE_MAP = {
    'Nom': '',           # İşaretsiz (unmarked) - FST'de tag yok
    'Acc': '+ACC',
    'Dat': '+DAT',
    'Loc': '+LOC',
    'Abl': '+ABL',
    'Gen': '+GEN',
    'Ins': '+INS',
    'Equ': '+EQU',       # -ca/-ce eşitlik
}

# ── 1C. Number (Sayı) Eşleme ──
NUMBER_MAP = {
    'Sing': '',          # Tekil - işaretsiz
    'Plur': '+PL',
}

# ── 1D. Person (Kişi) Eşleme ──
# Number + Person birleştirilerek FST person tag üretilir
PERSON_NUMBER_MAP = {
    ('Sing', '1'): '+1SG',
    ('Sing', '2'): '+2SG',
    ('Sing', '3'): '+3SG',   # Bazı bağlamlarda boş olabilir
    ('Plur', '1'): '+1PL',
    ('Plur', '2'): '+2PL',
    ('Plur', '3'): '+3PL',
}

# ── 1E. Tense (Zaman) Eşleme ──
TENSE_MAP = {
    'Past': '+PAST',
    'Pres': '',          # Şimdiki zaman - Aspect ile birlikte değerlendirilir
    'Fut':  '+FUT',
}

# ── 1F. Aspect (Görünüş) Eşleme ──
ASPECT_MAP = {
    'Perf': '',          # Perfective - genelde tense ile birlikte
    'Prog': '+PRES.CONT',  # -Iyor (şimdiki zaman sürerlik)
    'Hab':  '+AOR',      # Geniş zaman
    'Rapid':'+IVER',     # Tezlik fiili (nadiren)
}

# ── 1G. Mood (Kip) Eşleme ──
MOOD_MAP = {
    'Ind':  '',          # Bildirme kipi - varsayılan
    'Imp':  '+IMP',
    'Opt':  '+OPT',      # İstek kipi (-a/-e)
    'Des':  '+OPT',      # Dilek kipi (UD'de Desiderative = FST'de OPT)
    'Nec':  '+NEC',      # Gereklilik (-malı/-meli)
    'Pot':  '+ABIL',     # Yeterlilik (-abil/-ebil)
    'Cnd':  '+COND',     # Şart kipi (-sa/-se)
    'Gen':  '+COP.PRES', # Genelleyici -(I)dIr
    'CndGen':'+COND',    # Koşullu genelleyici (nadir)
}

# ── 1H. Voice (Çatı) Eşleme ──
VOICE_MAP = {
    'Pass': '+PASS',     # Edilgen
    'Cau':  '+CAUS',     # Ettirgen
    'Rfl':  '+REFL',     # Dönüşlü
    'Rcp':  '+RECIP',    # İşteş
    'CauPass':'+CAUS+PASS', # Ettirgen + Edilgen
}

# ── 1I. Polarity (Olumsuzluk) Eşleme ──
POLARITY_MAP = {
    'Pos': '',           # Olumlu - işaretsiz
    'Neg': '+NEG',
}

# ── 1J. Possessive (İyelik) Eşleme ──
# UD: Number[psor] + Person[psor] → FST: +POSS.XY
POSS_MAP = {
    ('Sing', '1'): '+POSS.1SG',
    ('Sing', '2'): '+POSS.2SG',
    ('Sing', '3'): '+POSS.3SG',
    ('Plur', '1'): '+POSS.1PL',
    ('Plur', '2'): '+POSS.2PL',
    ('Plur', '3'): '+POSS.3PL',
}

# ── 1K. Evident (Kanıtsallık) Eşleme ──
EVIDENT_MAP = {
    'Fh':  '',           # Birinci elden (firsthand) - varsayılan
    'Nfh': '+INFER',     # Duyulan geçmiş (-mIş)
}

# ── 1L. VerbForm (Fiil Biçimi) Eşleme ──
VERBFORM_MAP = {
    'Part':  '',         # Sıfat-fiil - bağlama göre analiz
    'Conv':  '+CVB',     # Zarf-fiil (belirli alt tip bağlama göre)
    'Vnoun': '',         # İsim-fiil (-mA, -mAk, -Iş)
    'Fin':   '',         # Çekimli fiil - varsayılan
}

# ── 1M. Derivational (Türetim) Eşleme ──
# UD'de doğrudan yok, ancak bazı NAdj etiketleri ipucu verir
# FST: +DER.lık, +DER.cı, +DER.sız, +DER.lı vb.
# Bu özellikler UD'den otomatik çıkarılamaz, lemma+XPOS'tan tahmin edilebilir

print('✅ Eşleme tabloları yüklendi.')
print(f'   UPOS eşleme: {len(UPOS_TO_FST_POS)} kategori')
print(f'   Case eşleme: {len(CASE_MAP)} durum')
print(f'   Tense eşleme: {len(TENSE_MAP)} zaman')
print(f'   Mood eşleme: {len(MOOD_MAP)} kip')
print(f'   Voice eşleme: {len(VOICE_MAP)} çatı')


## ⚠️ Belirsiz / Doğrudan Eşlenemeyen Özellikler

| Durum | UD Tarafı | FST Tarafı | Karar |
|-------|-----------|------------|-------|
| **AUX (yardımcı fiil)** | `AUX` (değil, i-) | `+COP.*` (copula eki) | AUX → COP altı etiketlerine dönüştürülür |
| **DET (belirteç)** | `DET` (bir, bu, o) | FST'de yok | DET kalır veya PRON'a atanır |
| **Aspect=Perf** | Bitmişlik | FST'de yok | Tense ile birlikte yorumlanır |
| **Polite=Infm** | Kayıt dışı | FST'de yok | İhmal edilir (bilgi kaybı) |
| **PronType** | Dem/Prs/Qes... | FST'de yok | İhmal, ama geri çevrilebilirlik kaybı |
| **3SG varsayılan** | Açık `Person=3` | FST bazen boş bırakır | Tutarlılık için +3SG yazılır |
| **NAdj (XPOS)** | ADJ/NOUN arası | NOUN veya ADJ | UPOS'a güvenilir |
| **Türetim ekleri** | UD'de `li/sız/lık/cı` | `+DER.*` | Lemma farkından çıkarım yapılır |
| **VerbForm=Part** | Sıfat-fiil | FST'de explicit yok | VERB+Part notasyonu eklenir |
| **VerbForm=Conv** | Zarf-fiil | `+CVB.*` | Alt tip belirsiz, genel +CVB |

### 🔴 FST Daha Zengin Olduğu Durumlar
- **+EMPH** (vurgulama): UD'de karşılığı yok → UD→FST'de üretilemez
- **+KI** (relativizer): UD'de `ADP` `ki` olarak ayrı token → birleştirme gerekir
- **+CVB.INCA/KEN/IP/MADAN**: UD sadece `Conv` der, FST alt tip belirtir
- **+DER.lık/cı/sız/lı**: UD'de morfolojik türetim belirtilmez

### 🟡 UD Daha Zengin Olduğu Durumlar
- **Polite=Infm/Form**: FST'de karşılık yok → ihmal
- **PronType=Prs/Dem/Qes**: FST zamirleri alt tiplere ayırmaz → ihmal
- **NumType=Card/Ord**: FST'de NUM tek kategori → ihmal


In [ ]:
# ============================================================
# BÖLÜM 2: map_ud_to_fst() ANA FONKSİYONU
# ============================================================

def parse_ud_feats(feats_str):
    """UD FEATS string'ini dict'e çevirir: 'Case=Acc|Number=Sing' → {'Case':'Acc', ...}"""
    if not feats_str or feats_str == '_':
        return {}
    result = {}
    for pair in feats_str.split('|'):
        if '=' in pair:
            k, v = pair.split('=', 1)
            result[k] = v
    return result


def map_ud_to_fst(lemma, upos, feats_dict, xpos=None):
    """
    UD anotasyonunu FST analiz string'ine dönüştürür.
    
    Args:
        lemma (str): UD lemma
        upos (str): UD UPOS etiketi
        feats_dict (dict): UD özellik sözlüğü {'Case':'Acc', ...}
        xpos (str): İsteğe bağlı XPOS etiketi (NAdj algılama için)
    
    Returns:
        str: FST formatında analiz string'i
        list: Uyarılar/bilgi kayıpları listesi
    """
    warnings = []
    tags = []
    
    # ── 1. POS Eşleme ──
    fst_pos = UPOS_TO_FST_POS.get(upos, 'UNK')
    if fst_pos == 'UNK':
        warnings.append(f'Bilinmeyen UPOS: {upos}')
    
    # AUX özel işlem: copula olarak eşle
    if upos == 'AUX':
        return _map_aux_to_fst(lemma, feats_dict, warnings)
    
    # PUNCT özel işlem
    if upos == 'PUNCT':
        punct_map = {'.': '+PUNCT.period', ',': '+PUNCT.comma', 
                     '?': '+PUNCT.question', '!': '+PUNCT.exclamation',
                     ':': '+PUNCT.colon', ';': '+PUNCT.semicolon',
                     '...': '+PUNCT.ellipsis', '-': '+PUNCT.dash'}
        suffix = punct_map.get(lemma, '+PUNCT')
        return f'{lemma}{suffix}', warnings
    
    # ── 2. Fiil Morfolojisi ──
    if fst_pos == 'VERB':
        return _map_verb_to_fst(lemma, feats_dict, warnings)
    
    # ── 3. İsim/Sıfat/Zamir Morfolojisi ──
    if fst_pos in ('NOUN', 'PROPN', 'ADJ', 'PRON', 'NUM', 'DET'):
        return _map_nominal_to_fst(lemma, fst_pos, feats_dict, warnings)
    
    # ── 4. Basit Kategoriler (ADV, CONJ, POSTP, INTERJ) ──
    return f'{lemma}+{fst_pos}', warnings


def _map_verb_to_fst(lemma, feats, warnings):
    """Fiil morfolojisini FST formatına dönüştürür."""
    parts = [f'{lemma}+VERB']
    
    # Çatı (Voice)
    voice = feats.get('Voice', '')
    if voice and voice in VOICE_MAP:
        parts.append(VOICE_MAP[voice])
    
    # Yeterlilik (Mood=Pot → +ABIL)
    mood = feats.get('Mood', 'Ind')
    if mood == 'Pot':
        parts.append('+ABIL')
    
    # Olumsuzluk (Polarity)
    polarity = feats.get('Polarity', 'Pos')
    neg_tag = POLARITY_MAP.get(polarity, '')
    if neg_tag:
        parts.append(neg_tag)
    
    # VerbForm kontrol (Part, Conv, Vnoun)
    verb_form = feats.get('VerbForm', '')
    
    if verb_form == 'Conv':
        # Zarf-fiil - UD alt tip belirtmez, genel CVB
        parts.append('+CVB')
        warnings.append('CVB alt tipi (INCA/KEN/IP/MADAN) UD\'den çıkarılamaz')
        return ''.join(parts), warnings
    
    if verb_form == 'Part':
        # Sıfat-fiil
        parts.append('+PART')
    
    if verb_form == 'Vnoun':
        # İsim-fiil
        parts.append('+VNOUN')
    
    # Zaman + Görünüş (Tense + Aspect)
    tense = feats.get('Tense', '')
    aspect = feats.get('Aspect', '')
    evident = feats.get('Evident', '')
    
    # Duyulan geçmiş (-mIş)
    if evident == 'Nfh':
        parts.append('+INFER')
    elif aspect == 'Prog':
        parts.append('+PRES.CONT')
    elif aspect == 'Hab':
        parts.append('+AOR')
    elif tense == 'Past':
        parts.append('+PAST')
    elif tense == 'Fut':
        parts.append('+FUT')
    elif tense == 'Pres' and aspect == 'Perf':
        pass  # Geniş zaman / şimdiki bağlamda zaman eki yok
    
    # Kip (Mood) - Pot zaten yukarıda işlendi
    if mood and mood not in ('Ind', 'Pot'):
        mood_tag = MOOD_MAP.get(mood, '')
        if mood_tag:
            parts.append(mood_tag)
    
    # Kişi + Sayı
    number = feats.get('Number', '')
    person = feats.get('Person', '')
    if number and person:
        pn_tag = PERSON_NUMBER_MAP.get((number, person), '')
        if pn_tag:
            parts.append(pn_tag)
    
    # İyelik (possessor - fiil sıfat-fiillerinde)
    npsor = feats.get('Number[psor]', '')
    ppsor = feats.get('Person[psor]', '')
    if npsor and ppsor:
        poss_tag = POSS_MAP.get((npsor, ppsor), '')
        if poss_tag:
            parts.append(poss_tag)
    
    # Durum (fiil-isimleşmelerde case olabilir)
    case = feats.get('Case', '')
    if case and case != 'Nom':
        case_tag = CASE_MAP.get(case, '')
        if case_tag:
            parts.append(case_tag)
    
    # Polite bilgi kaybı
    if 'Polite' in feats:
        warnings.append(f'Polite={feats["Polite"]} FST\'de karşılığı yok')
    
    return ''.join(parts), warnings


def _map_nominal_to_fst(lemma, fst_pos, feats, warnings):
    """İsim/Sıfat/Zamir morfolojisini FST formatına dönüştürür."""
    parts = [f'{lemma}+{fst_pos}']
    
    # Çoğul
    number = feats.get('Number', 'Sing')
    if number == 'Plur':
        parts.append('+PL')
    
    # İyelik (Possessive)
    npsor = feats.get('Number[psor]', '')
    ppsor = feats.get('Person[psor]', '')
    if npsor and ppsor:
        poss_tag = POSS_MAP.get((npsor, ppsor), '')
        if poss_tag:
            parts.append(poss_tag)
    
    # Durum (Case)
    case = feats.get('Case', '')
    if case:
        case_tag = CASE_MAP.get(case, '')
        if case_tag:
            parts.append(case_tag)
    
    # PronType bilgi kaybı
    if 'PronType' in feats:
        warnings.append(f'PronType={feats["PronType"]} FST\'de yok')
    
    return ''.join(parts), warnings


def _map_aux_to_fst(lemma, feats, warnings):
    """AUX (yardımcı fiil/copula) eşlemesi."""
    # Türkçe'de AUX genelde copula "i-" veya "değil"
    parts = [f'{lemma}+AUX']
    
    tense = feats.get('Tense', '')
    mood = feats.get('Mood', '')
    evident = feats.get('Evident', '')
    polarity = feats.get('Polarity', '')
    
    if polarity == 'Neg':
        parts.append('+NEG')
    
    if evident == 'Nfh':
        parts.append('+COP.EVID')
    elif tense == 'Past':
        parts.append('+COP.PAST')
    elif mood == 'Gen':
        parts.append('+COP.PRES')
    elif mood == 'Cnd':
        parts.append('+COP.COND')
    else:
        parts.append('+COP')
    
    # Kişi
    number = feats.get('Number', '')
    person = feats.get('Person', '')
    if number and person:
        pn_tag = PERSON_NUMBER_MAP.get((number, person), '')
        if pn_tag:
            parts.append(pn_tag)
    
    return ''.join(parts), warnings


print('✅ map_ud_to_fst() fonksiyonu hazır.')


## 🧪 Simülasyon: 10 UD Token → FST Çıktısı

Aşağıda UD Turkish-IMST'den seçilmiş 10 token üzerinde dönüşüm simülasyonu yapılmaktadır.


In [ ]:
# ============================================================
# BÖLÜM 3: 10 ÖRNEK UD TOKEN SİMÜLASYONU
# ============================================================

import pandas as pd

# UD Turkish-IMST'den gerçek örnekler
test_tokens = [
    # (FORM, LEMMA, UPOS, FEATS_str)
    ('seviyorum', 'sev', 'VERB',
     'Aspect=Prog|Mood=Ind|Number=Sing|Person=1|Polarity=Pos|Polite=Infm|Tense=Pres'),
    ('yaşlanmayacağız', 'yaşlan', 'VERB',
     'Aspect=Perf|Mood=Ind|Number=Plur|Person=1|Polarity=Neg|Tense=Fut'),
    ('Seni', 'sen', 'PRON',
     'Case=Acc|Number=Sing|Person=2|PronType=Prs'),
    ('limandan', 'liman', 'NOUN',
     'Case=Abl|Number=Sing|Person=3'),
    ('Aşkımız', 'aşk', 'NOUN',
     'Case=Nom|Number=Sing|Number[psor]=Plur|Person=3|Person[psor]=1'),
    ('eskimeyecek', 'eski', 'VERB',
     'Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Polarity=Neg|Tense=Fut'),
    ('demiş', 'de', 'VERB',
     'Aspect=Perf|Evident=Nfh|Mood=Ind|Number=Sing|Person=3|Polarity=Pos|Tense=Past'),
    ('öğrencisiyle', 'öğrenci', 'NOUN',
     'Case=Ins|Number=Sing|Number[psor]=Sing|Person=3|Person[psor]=3'),
    ('çıkınca', 'çık', 'VERB',
     'Aspect=Perf|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Conv'),
    ('bakın', 'bak', 'VERB',
     'Aspect=Perf|Mood=Imp|Number=Plur|Person=2|Polarity=Pos|Tense=Pres'),
]

print('🧪 UD → FST SİMÜLASYON SONUÇLARI')
print('=' * 90)
print(f'{"FORM":<20} {"LEMMA":<12} {"UPOS":<8} {"FST Çıktısı":<35} {"Uyarılar"}')
print('-' * 90)

results = []
for form, lemma, upos, feats_str in test_tokens:
    feats_dict = parse_ud_feats(feats_str)
    fst_output, warns = map_ud_to_fst(lemma, upos, feats_dict)
    warn_str = '; '.join(warns) if warns else '-'
    print(f'{form:<20} {lemma:<12} {upos:<8} {fst_output:<35} {warn_str}')
    results.append({
        'Form': form, 'Lemma': lemma, 'UPOS': upos,
        'FEATS': feats_str, 'FST_Output': fst_output, 'Warnings': warn_str
    })

print('\n✅ Simülasyon tamamlandı.')

# DataFrame olarak da göster
df_sim = pd.DataFrame(results)
df_sim


In [ ]:
# ============================================================
# BÖLÜM 4: CoNLL-U DOSYASINI OKUYUP TOPLU FST DÖNÜŞÜMÜ
# ============================================================

import os

def parse_conllu(filepath):
    """CoNLL-U dosyasını parse eder, token listesi döndürür."""
    tokens = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split('\t')
            if len(parts) < 10:
                continue
            # Multi-word token satırlarını atla (1-2 gibi)
            if '-' in parts[0] or '.' in parts[0]:
                continue
            tokens.append({
                'id': parts[0],
                'form': parts[1],
                'lemma': parts[2],
                'upos': parts[3],
                'xpos': parts[4],
                'feats': parts[5],
            })
    return tokens


def batch_ud_to_fst(tokens):
    """Token listesini toplu olarak FST formatına dönüştürür."""
    results = []
    warn_counts = {}
    
    for tok in tokens:
        feats_dict = parse_ud_feats(tok['feats'])
        fst_out, warns = map_ud_to_fst(
            tok['lemma'], tok['upos'], feats_dict, tok.get('xpos')
        )
        results.append({
            'form': tok['form'],
            'lemma': tok['lemma'],
            'upos': tok['upos'],
            'feats': tok['feats'],
            'fst_analysis': fst_out,
            'warnings': warns,
        })
        for w in warns:
            warn_counts[w] = warn_counts.get(w, 0) + 1
    
    return results, warn_counts


# ── Dosya yolunu ayarla ──
# Yerel ortam için:
CONLLU_PATH = os.path.join('..', '..', 'tr_imst-ud-test.conllu')
# Colab için alternatif:
# CONLLU_PATH = '/content/drive/MyDrive/FSTurk/tr_imst-ud-test.conllu'

if os.path.exists(CONLLU_PATH):
    tokens = parse_conllu(CONLLU_PATH)
    print(f'📦 {len(tokens)} token okundu.')
    
    fst_results, warn_summary = batch_ud_to_fst(tokens)
    
    # İstatistikler
    print(f'✅ {len(fst_results)} token FST formatına dönüştürüldü.')
    print(f'\n⚠️ Uyarı Özeti ({len(warn_summary)} farklı uyarı tipi):')
    for w, count in sorted(warn_summary.items(), key=lambda x:-x[1])[:10]:
        print(f'   {count:>5}x | {w}')
    
    # İlk 20 örneği göster
    print(f'\n📋 İlk 20 Dönüşüm Örneği:')
    print(f'{"FORM":<20} {"UPOS":<8} {"FST Analizi"}')
    print('-' * 60)
    for r in fst_results[:20]:
        print(f'{r["form"]:<20} {r["upos"]:<8} {r["fst_analysis"]}')
else:
    print(f'❌ CoNLL-U dosyası bulunamadı: {CONLLU_PATH}')
    print('   Lütfen dosya yolunu kontrol edin.')


In [ ]:
# ============================================================
# BÖLÜM 5: FST ANALİZLERİNİ DIŞA AKTARMA
# ============================================================

import json

def export_fst_analyses(results, output_path):
    """FST analizlerini JSON formatında kaydeder."""
    export_data = []
    for r in results:
        export_data.append({
            'form': r['form'],
            'lemma': r['lemma'],
            'upos': r['upos'],
            'ud_feats': r['feats'],
            'fst_analysis': r['fst_analysis'],
            'warnings': r['warnings'],
        })
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    print(f'💾 {len(export_data)} analiz kaydedildi: {output_path}')


def export_fst_plain(results, output_path):
    """FST analizlerini düz metin formatında kaydeder (FST test girişi için)."""
    with open(output_path, 'w', encoding='utf-8') as f:
        for r in results:
            f.write(f'{r["form"]}\t{r["fst_analysis"]}\n')
    
    print(f'📝 Düz metin çıktısı: {output_path}')


# Kaydet
if 'fst_results' in dir() and fst_results:
    # JSON formatı
    export_fst_analyses(fst_results, 'ud_to_fst_mapped.json')
    # Düz metin
    export_fst_plain(fst_results, 'ud_to_fst_mapped.txt')
    print('\n✅ Tüm çıktılar hazır!')
else:
    print('⚠️ Önce CoNLL-U dosyasını yükleyin (Bölüm 4).')


In [ ]:
# ============================================================
# BÖLÜM 6: DOĞRULAMA ve İSTATİSTİKLER
# ============================================================

if 'fst_results' in dir() and fst_results:
    from collections import Counter
    
    # POS dağılımı
    pos_counter = Counter(r['upos'] for r in fst_results)
    print('📊 UPOS Dağılımı:')
    for pos, count in pos_counter.most_common():
        fst_eq = UPOS_TO_FST_POS.get(pos, '?')
        print(f'   {pos:<8} → {fst_eq:<8} : {count:>5} token')
    
    # Uyarısız token oranı
    clean = sum(1 for r in fst_results if not r['warnings'])
    total = len(fst_results)
    print(f'\n🎯 Temiz Dönüşüm Oranı: {clean}/{total} ({100*clean/total:.1f}%)')
    
    # Fiil kipi dağılımı
    verb_moods = []
    for r in fst_results:
        if r['upos'] == 'VERB':
            fd = parse_ud_feats(r['feats'])
            m = fd.get('Mood', 'Ind')
            verb_moods.append(m)
    mood_counter = Counter(verb_moods)
    print(f'\n📊 Fiil Kipi Dağılımı:')
    for mood, count in mood_counter.most_common():
        fst_eq = MOOD_MAP.get(mood, '?')
        print(f'   {mood:<10} → {fst_eq:<12} : {count:>5} fiil')
    
    print('\n✅ Doğrulama tamamlandı.')
else:
    print('⚠️ Önce CoNLL-U dosyasını yükleyin (Bölüm 4).')


In [ ]:
# ============================================================
# PATCH A: Tense x Aspect DETERMINISTIK MATRIS
# ============================================================
# Risk: UD'de Tense ve Aspect ayri ozellikler.
#        FST'te birlestik tag (PRES.CONT, AOR, PAST vs.)
#        Her kombinasyon icin tek bir FST tag uretilmeli.

# Tam deterministik matris: (Tense, Aspect, Evident) -> FST tag
TENSE_ASPECT_MATRIX = {
    # (Tense, Aspect, Evident) -> FST tense/aspect tag
    
    # --- PRESENT ---
    ('Pres', 'Prog',  ''):    '+PRES.CONT',   # -Iyor (simdi devam)
    ('Pres', 'Hab',   ''):    '+AOR',          # -(I)r (genis zaman)
    ('Pres', 'Perf',  ''):    '',              # isaretsiz (isim cumlesi vb.)
    ('Pres', '',      ''):    '',              # bare present
    
    # --- PAST ---
    ('Past', 'Perf',  ''):    '+PAST',         # -DI (gorulmus gecmis)
    ('Past', 'Perf',  'Nfh'): '+INFER',        # -mIs (duyulan gecmis)
    ('Past', 'Prog',  ''):    '+PAST.CONT',    # -Iyordu (gecmis devam)
    ('Past', 'Hab',   ''):    '+AOR+PAST',     # -Irdi (genis zaman + gecmis)
    ('Past', '',      ''):    '+PAST',         # fallback past
    ('Past', '',      'Nfh'): '+INFER',        # fallback infer
    
    # --- FUTURE ---
    ('Fut',  'Perf',  ''):    '+FUT',          # -(y)AcAk
    ('Fut',  'Prog',  ''):    '+FUT',          # nadir, fallback FUT
    ('Fut',  '',      ''):    '+FUT',          # fallback future
    
    # --- EVIDENTIAL (no explicit tense) ---
    ('',     'Perf',  'Nfh'): '+INFER',        # -mIs (tense bos)
    ('',     '',      'Nfh'): '+INFER',        # fallback infer
    
    # --- EMPTY (tense/aspect bos) ---
    ('',     'Prog',  ''):    '+PRES.CONT',    # -Iyor (tense bos ama prog)
    ('',     'Hab',   ''):    '+AOR',          # -(I)r (tense bos ama hab)
    ('',     'Perf',  ''):    '',              # bare perf
    ('',     '',      ''):    '',              # tamamen bos
}


def resolve_tense_aspect(feats):
    """
    Tense x Aspect x Evident uclusunden deterministik FST tag uretir.
    
    Returns:
        str: FST tense/aspect tag(lar)i
        str: Uyari mesaji (bos string = uyari yok)
    """
    tense = feats.get('Tense', '')
    aspect = feats.get('Aspect', '')
    evident = feats.get('Evident', '')
    
    key = (tense, aspect, evident)
    
    if key in TENSE_ASPECT_MATRIX:
        return TENSE_ASPECT_MATRIX[key], ''
    
    # Fallback: bilinen parcalari dene
    # Evident > Aspect > Tense oncelik sirasi
    if evident == 'Nfh':
        return '+INFER', f'Fallback INFER: {key}'
    if aspect == 'Prog':
        return '+PRES.CONT', f'Fallback PROG: {key}'
    if aspect == 'Hab':
        return '+AOR', f'Fallback HAB: {key}'
    if tense == 'Past':
        return '+PAST', f'Fallback PAST: {key}'
    if tense == 'Fut':
        return '+FUT', f'Fallback FUT: {key}'
    
    return '', f'Bilinmeyen Tense/Aspect/Evident kombinasyonu: {key}'


# Test: Tum matris entries
print('Tense x Aspect DETERMINISTIC MATRIS TESTI')
print('=' * 70)
test_combos = [
    {'Tense': 'Pres', 'Aspect': 'Prog'},                    # -Iyor
    {'Tense': 'Past', 'Aspect': 'Perf'},                    # -DI
    {'Tense': 'Past', 'Aspect': 'Perf', 'Evident': 'Nfh'},  # -mIs
    {'Tense': 'Past', 'Aspect': 'Prog'},                    # -Iyordu
    {'Tense': 'Past', 'Aspect': 'Hab'},                     # -Irdi
    {'Tense': 'Fut',  'Aspect': 'Perf'},                    # -(y)AcAk
    {'Tense': 'Pres', 'Aspect': 'Hab'},                     # -(I)r
    {'Tense': 'Pres', 'Aspect': 'Perf'},                    # bare
]

for combo in test_combos:
    tag, warn = resolve_tense_aspect(combo)
    t = combo.get('Tense','-')
    a = combo.get('Aspect','-')
    e = combo.get('Evident','-')
    status = 'OK' if not warn else f'WARN: {warn}'
    print(f'  T={t:<5} A={a:<5} E={e:<4} -> {tag or "(bos)":<15} {status}')

print('\nDeterministik mi? EVET - her (T,A,E) ikilisi tek sonuc verir.')


In [ ]:
# ============================================================
# PATCH B: Person DEFAULT + GUNCELLENMIS map_ud_to_fst
# ============================================================
# Risk: UD'de bazen Person yok (implicit 3SG).
#        FST'de +3SG zorunlu mu?
#        Karar: FST motorunda +3SG bazen bos, bazen explicit.
#        Tutarlilik icin: Person yoksa VE Number yoksa -> +3SG default

def resolve_person_number(feats, is_verb=False):
    """
    Person x Number eslesmesi, eksik degerler icin default kurallar.
    
    Kurallar:
    - Person + Number varsa: normal esleme
    - Person var, Number yok: Sing varsayilir
    - Number var, Person yok: 3 varsayilir
    - Ikisi de yok + fiilse: +3SG (Turkce default)
    - Ikisi de yok + isimse: bos (isaretsiz)
    """
    number = feats.get('Number', '')
    person = feats.get('Person', '')
    
    # Ikisi de var -> normal
    if number and person:
        return PERSON_NUMBER_MAP.get((number, person), ''), ''
    
    # Person var, Number yok -> Sing varsayilir
    if person and not number:
        tag = PERSON_NUMBER_MAP.get(('Sing', person), '')
        return tag, f'Number eksik, Sing varsayildi (Person={person})'
    
    # Number var, Person yok -> 3 varsayilir
    if number and not person:
        tag = PERSON_NUMBER_MAP.get((number, '3'), '')
        return tag, f'Person eksik, 3 varsayildi (Number={number})'
    
    # Ikisi de yok
    if is_verb:
        return '+3SG', 'Person+Number eksik, fiil icin +3SG varsayildi'
    
    return '', ''  # Isimler icin isaretsiz


# ── GUNCELLENMIS FIIL ESLEME FONKSIYONU ──
def _map_verb_to_fst_v2(lemma, feats, warnings):
    # Fiil morfolojisi - v2: deterministik Tense*Aspect + Person default.
    parts = [f'{lemma}+VERB']
    
    # 1. Cati (Voice)
    voice = feats.get('Voice', '')
    if voice and voice in VOICE_MAP:
        parts.append(VOICE_MAP[voice])
    
    # 2. Yeterlilik (Mood=Pot -> +ABIL)
    mood = feats.get('Mood', 'Ind')
    if mood == 'Pot':
        parts.append('+ABIL')
    
    # 3. Olumsuzluk
    polarity = feats.get('Polarity', 'Pos')
    neg_tag = POLARITY_MAP.get(polarity, '')
    if neg_tag:
        parts.append(neg_tag)
    
    # 4. VerbForm (Conv, Part, Vnoun)
    verb_form = feats.get('VerbForm', '')
    if verb_form == 'Conv':
        parts.append('+CVB')
        warnings.append('CVB alt tipi (INCA/KEN/IP/MADAN) UD\'den cikarilmaz')
        return ''.join(parts), warnings
    if verb_form == 'Part':
        parts.append('+PART')
    if verb_form == 'Vnoun':
        parts.append('+VNOUN')
    
    # 5. Tense x Aspect x Evident -> DETERMINISTIK MATRIS
    ta_tag, ta_warn = resolve_tense_aspect(feats)
    if ta_tag:
        parts.append(ta_tag)
    if ta_warn:
        warnings.append(ta_warn)
    
    # 6. Kip (Mood) - Ind ve Pot zaten islendi
    if mood and mood not in ('Ind', 'Pot'):
        mood_tag = MOOD_MAP.get(mood, '')
        if mood_tag:
            parts.append(mood_tag)
    
    # 7. Kisi x Sayi -> DEFAULT KURALLI
    pn_tag, pn_warn = resolve_person_number(feats, is_verb=True)
    if pn_tag:
        parts.append(pn_tag)
    if pn_warn:
        warnings.append(pn_warn)
    
    # 8. Iyelik (possessor - sifat-fiillerde)
    npsor = feats.get('Number[psor]', '')
    ppsor = feats.get('Person[psor]', '')
    if npsor and ppsor:
        poss_tag = POSS_MAP.get((npsor, ppsor), '')
        if poss_tag:
            parts.append(poss_tag)
    
    # 9. Durum (fiil-isimlesmelerinde case)
    case = feats.get('Case', '')
    if case and case != 'Nom':
        case_tag = CASE_MAP.get(case, '')
        if case_tag:
            parts.append(case_tag)
    
    # 10. Polite bilgi kaybi
    if 'Polite' in feats:
        warnings.append(f'Polite={feats["Polite"]} FST\'de yok')
    
    return ''.join(parts), warnings


# ── ANA FONKSIYONU GUNCELLE ──
def map_ud_to_fst_v2(lemma, upos, feats_dict, xpos=None):
    """map_ud_to_fst v2: deterministik T*A matrisi + Person default."""
    warnings = []
    fst_pos = UPOS_TO_FST_POS.get(upos, 'UNK')
    if fst_pos == 'UNK':
        warnings.append(f'Bilinmeyen UPOS: {upos}')
    
    if upos == 'AUX':
        return _map_aux_to_fst(lemma, feats_dict, warnings)
    if upos == 'PUNCT':
        punct_map = {'.': '+PUNCT.period', ',': '+PUNCT.comma',
                     '?': '+PUNCT.question', '!': '+PUNCT.exclamation',
                     ':': '+PUNCT.colon', ';': '+PUNCT.semicolon',
                     '...': '+PUNCT.ellipsis', '-': '+PUNCT.dash'}
        return f'{lemma}{punct_map.get(lemma, "+PUNCT")}', warnings
    if fst_pos == 'VERB':
        return _map_verb_to_fst_v2(lemma, feats_dict, warnings)
    if fst_pos in ('NOUN', 'PROPN', 'ADJ', 'PRON', 'NUM', 'DET'):
        return _map_nominal_to_fst(lemma, fst_pos, feats_dict, warnings)
    return f'{lemma}+{fst_pos}', warnings


# ── V2 TEST ──
print('\nV2 TEST: Past+Prog ve Person-eksik durumlar')
print('=' * 70)
v2_tests = [
    ('seviyordum', 'sev', 'VERB',
     'Aspect=Prog|Mood=Ind|Number=Sing|Person=1|Polarity=Pos|Tense=Past'),
    ('gelirdi', 'gel', 'VERB',
     'Aspect=Hab|Mood=Ind|Number=Sing|Person=3|Polarity=Pos|Tense=Past'),
    ('patlayan', 'patla', 'VERB',
     'Aspect=Perf|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Part'),
    ('cikinca', 'cik', 'VERB',
     'Aspect=Perf|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Conv'),
    ('yapmak', 'yap', 'VERB',
     'Aspect=Perf|Case=Nom|Mood=Ind|Polarity=Pos|Tense=Pres|VerbForm=Vnoun'),
]

for form, lemma, upos, feats_str in v2_tests:
    fd = parse_ud_feats(feats_str)
    fst, warns = map_ud_to_fst_v2(lemma, upos, fd)
    w = '; '.join(warns) if warns else '-'
    print(f'  {form:<20} -> {fst:<30} | {w}')


## 🔴 KRITIK: Derivational Boundary (Turetim Siniri) Kaybi

### Problem
UD Turkish derivasyonel morfolojiyi cogu zaman **acik olarak belirtmez**.

| Form | UD Analizi | FST Beklenen | Sorun |
|------|-----------|-------------|-------|
| yuzlum | `yuz+NOUN+Poss1Sg` | `yuz+NOUN+DER.lu+POSS.1SG` | UD: DER.lu KAYIP |
| kitaplik | `kitaplik+NOUN` | `kitap+NOUN+DER.lik` | UD: lemma zaten turetilmis |
| isci | `isci+NOUN` | `is+NOUN+DER.ci` | UD: lemma turetilmis |
| evsiz | `evsiz+ADJ` | `ev+NOUN+DER.siz` | UD: POS bile degisti |

### Etki
- **FST candidate ile UD gold birebir eslesMEYECEK** turetilmis kelimelerde
- Bu, coverage oranini dusurur
- Bigram model icin gold standard olusturmayi zorlastirir

### Cozum Stratejisi
1. **Lemma karsilastirmasi**: UD lemma vs FST lemma farki varsa → turetim eki var 
2. **Suffix sezgisel tespiti**: UD lemma sonunda `-lik/-lik/-siz/-ci/-li` varsa DER tag ekle
3. **Kabul**: Tam geri cevrilebilirlik turetim icin MUMKUN DEGIL, bu bilinen sinir
4. **Metric**: Coverage olcumunde turetilmis kelimeleri ayri raporla


In [ ]:
# ============================================================
# BOLUM 7: FST COVERAGE DEGERLENDIRME
# ============================================================

def evaluate_fst_coverage(ud_fst_mapped, fst_analyzer_func):
    """
    UD->FST esleme sonuclarini gercek FST analyzer ile karsilastirir.
    Bu oran senin gercek ceiling'in.
    
    Args:
        ud_fst_mapped: list of dict, her biri:
            {'form': str, 'fst_analysis': str, ...}
        fst_analyzer_func: callable, surface -> list[str]
            Ornek: lambda w: analyze_word(w, analyzer)
    
    Returns:
        dict: coverage istatistikleri
    """
    total = 0
    exact_match = 0
    in_candidates = 0
    no_analysis = 0
    mismatches = []
    
    for token in ud_fst_mapped:
        surface = token['form']
        gold_fst = token['fst_analysis']
        
        # PUNCT atla
        if '+PUNCT' in gold_fst:
            continue
        
        total += 1
        
        try:
            candidates = fst_analyzer_func(surface.lower())
        except Exception:
            candidates = []
        
        if not candidates or (len(candidates)==1 and 'No analysis' in candidates[0]):
            no_analysis += 1
            mismatches.append({
                'form': surface, 'gold': gold_fst,
                'candidates': [], 'type': 'NO_ANALYSIS'
            })
            continue
        
        # Exact match
        if gold_fst in candidates:
            exact_match += 1
            in_candidates += 1
        else:
            # Pattern match: lemma+POS kismi eslesiyor mu?
            gold_prefix = gold_fst.split('+')[0] + '+' + gold_fst.split('+')[1] if '+' in gold_fst else gold_fst
            partial = any(c.startswith(gold_prefix) for c in candidates)
            if partial:
                in_candidates += 1
            
            mismatches.append({
                'form': surface, 'gold': gold_fst,
                'candidates': candidates[:3],
                'type': 'PARTIAL_MATCH' if partial else 'MISMATCH'
            })
    
    results = {
        'total': total,
        'exact_match': exact_match,
        'exact_match_pct': 100 * exact_match / total if total else 0,
        'in_candidates': in_candidates,
        'in_candidates_pct': 100 * in_candidates / total if total else 0,
        'no_analysis': no_analysis,
        'no_analysis_pct': 100 * no_analysis / total if total else 0,
        'mismatch_samples': mismatches[:20],
    }
    
    print(f'FST COVERAGE RAPORU')
    print(f'={"="*50}')
    print(f'Toplam token (PUNCT haric): {total}')
    print(f'Exact match:               {exact_match} ({results["exact_match_pct"]:.1f}%)')
    print(f'Candidate icinde:          {in_candidates} ({results["in_candidates_pct"]:.1f}%)')
    print(f'FST analiz yok:            {no_analysis} ({results["no_analysis_pct"]:.1f}%)')
    
    if mismatches:
        print(f'\nIlk 10 uyumsuzluk:')
        for m in mismatches[:10]:
            print(f'  {m["form"]:<20} gold={m["gold"]:<30} candidates={m["candidates"][:2]} [{m["type"]}]')
    
    return results


# ── Kullanim ornegi (FST analyzer yuklu degilse simule et) ──
print('evaluate_fst_coverage() fonksiyonu hazir.')
print('\nKullanim:')
print('  # FST analyzer yuklendikten sonra:')
print('  # results = evaluate_fst_coverage(fst_results, ')
print('  #     lambda w: analyze_word(w, analyzer))')
print('  # Bu oran senin gercek ceiling\'in.')


In [ ]:
# ============================================================
# BOLUM 8: PATTERN-LEVEL STATE (Bigram Model icin)
# ============================================================
# 
# Problem: Full FST string cok sparse olur.
#   Ornek: "sev+VERB+PRES.CONT+1SG" = unique string
#          "git+VERB+PAST+3SG" = baska unique string
#          Bunlar arasinda bigram ogrenemez.
# 
# Cozum: POS+FEATURE_PATTERN level state kullan.
#   "sev+VERB+PRES.CONT+1SG" -> state = "VERB+PRES.CONT+1SG"
#   "git+VERB+PAST+3SG"      -> state = "VERB+PAST+3SG"
#   Boylece lemma-bagimsiz pattern'lar ogrenilebilir.

def extract_fst_pattern(fst_analysis):
    """
    FST analiz string'inden lemma-bagimsiz pattern cikarir.
    
    Ornek:
        'sev+VERB+PRES.CONT+1SG' -> 'VERB+PRES.CONT+1SG'
        'ev+NOUN+PL+ABL'         -> 'NOUN+PL+ABL'
        'liman+NOUN+ABL'         -> 'NOUN+ABL'
    """
    if '+' not in fst_analysis:
        return fst_analysis
    
    parts = fst_analysis.split('+')
    # ilk parca lemma, geri kalani pattern
    return '+'.join(parts[1:])


def build_pattern_bigrams(fst_results):
    """
    FST sonuclarindan pattern-level bigram sayaclari olusturur.
    
    Returns:
        dict: {(prev_pattern, curr_pattern): count}
        dict: {pattern: count} unigram sayaclari
    """
    from collections import Counter
    
    bigrams = Counter()
    unigrams = Counter()
    
    prev_pattern = 'START'
    
    for r in fst_results:
        curr_pattern = extract_fst_pattern(r['fst_analysis'])
        
        unigrams[curr_pattern] += 1
        bigrams[(prev_pattern, curr_pattern)] += 1
        
        # Cumle sonu -> reset
        if curr_pattern.startswith('PUNCT'):
            prev_pattern = 'START'
        else:
            prev_pattern = curr_pattern
    
    return dict(bigrams), dict(unigrams)


# ── TEST ──
if 'fst_results' in dir() and fst_results:
    # V2 ile yeniden donustur
    fst_results_v2 = []
    for tok in tokens:
        fd = parse_ud_feats(tok['feats'])
        fst_out, warns = map_ud_to_fst_v2(tok['lemma'], tok['upos'], fd)
        fst_results_v2.append({
            'form': tok['form'], 'lemma': tok['lemma'],
            'upos': tok['upos'], 'feats': tok['feats'],
            'fst_analysis': fst_out, 'warnings': warns,
        })
    
    bigrams, unigrams = build_pattern_bigrams(fst_results_v2)
    
    print('PATTERN-LEVEL STATE ANALIZI')
    print('=' * 60)
    print(f'Unique pattern (state) sayisi: {len(unigrams)}')
    print(f'Unique bigram sayisi:          {len(bigrams)}')
    
    # En sik pattern'lar
    print(f'\nEn sik 15 pattern:')
    for pattern, count in sorted(unigrams.items(), key=lambda x: -x[1])[:15]:
        print(f'  {count:>5}x  {pattern}')
    
    # En sik bigram'lar  
    print(f'\nEn sik 15 bigram (gecis):')
    for (p1, p2), count in sorted(bigrams.items(), key=lambda x: -x[1])[:15]:
        print(f'  {count:>5}x  {p1:<25} -> {p2}')
    
    # Sparsity karsilastirmasi
    full_strings = set(r['fst_analysis'] for r in fst_results_v2)
    print(f'\nSparsity Karsilastirmasi:')
    print(f'  Full FST string (unique):  {len(full_strings)}')
    print(f'  Pattern-level (unique):    {len(unigrams)}')
    print(f'  Indirgeme orani:           {len(full_strings)/max(len(unigrams),1):.1f}x')
    print(f'\nPattern-level state kullanimi {len(full_strings)/max(len(unigrams),1):.1f}x daha az sparse.')
else:
    print('Once CoNLL-U dosyasini yukleyin (Bolum 4).')

print('\n\nExtract ornekleri:')
examples = ['sev+VERB+PRES.CONT+1SG', 'ev+NOUN+PL+ABL', 'liman+NOUN+ABL',
            'git+VERB+NEG+FUT+1PL', 'bak+VERB+IMP+2PL']
for ex in examples:
    print(f'  {ex:<35} -> {extract_fst_pattern(ex)}')


In [ ]:
# ============================================================
# BOLUM 9: COVERAGE TESTI (Yapisal Analiz)
# ============================================================
# pynini gerektirmez - mapped JSON uzerinde calisir.
# Gercek FST coverage icin Colab'da Bolum 7'yi kullanin.

import json
import os
from collections import Counter, defaultdict

# ── Mapped veriyi yukle ──
MAPPED_FILE = 'ud_to_fst_mapped.json'
if not os.path.exists(MAPPED_FILE):
    MAPPED_FILE = os.path.join(os.path.dirname(os.path.abspath('.')), MAPPED_FILE)

with open(MAPPED_FILE, 'r', encoding='utf-8') as f:
    mapped_data = json.load(f)

print(f'Yuklenen token sayisi: {len(mapped_data)}')

# ── Coverage Metrikleri ──
total = 0
valid_fst = 0         # Gecerli FST format (lemma+POS+...)
has_warnings = 0      # Uyari iceren
punct_count = 0
empty_feats = 0       # Bos features
pos_coverage = Counter()    # UPOS bazinda coverage
warning_types = Counter()   # Uyari tipleri
fst_tag_depth = Counter()   # FST tag derinligi (+ sayisi)

for tok in mapped_data:
    fst = tok['fst_analysis']
    warns = tok.get('warnings', [])
    upos = tok.get('upos', '?')
    
    # PUNCT atla
    if 'PUNCT' in fst:
        punct_count += 1
        continue
    
    total += 1
    
    # Gecerli FST kontrolu: en az lemma+POS icermeli
    if '+' in fst:
        valid_fst += 1
        tag_count = fst.count('+')
        fst_tag_depth[tag_count] += 1
    
    if warns:
        has_warnings += 1
        for w in warns:
            warning_types[w] += 1
    
    if tok.get('ud_feats', '') in ('', '_'):
        empty_feats += 1
    
    pos_coverage[upos] += 1

# ── RAPOR ──
print('\n' + '=' * 60)
print('COVERAGE RAPORU')
print('=' * 60)
print(f'Toplam token (PUNCT haric): {total}')
print(f'Noktalama:                  {punct_count}')
print(f'Gecerli FST format:         {valid_fst} ({100*valid_fst/total:.1f}%)')
print(f'Uyari iceren:               {has_warnings} ({100*has_warnings/total:.1f}%)')
print(f'Temiz donusum:              {total-has_warnings} ({100*(total-has_warnings)/total:.1f}%)')
print(f'Bos UD features:            {empty_feats}')

print(f'\nUPOS Bazinda Dagilim:')
for upos, count in pos_coverage.most_common():
    pct = 100 * count / total
    bar = '#' * int(pct / 2)
    print(f'  {upos:<8} {count:>5} ({pct:>5.1f}%) {bar}')

print(f'\nFST Tag Derinligi (+ sayisi):')
for depth, count in sorted(fst_tag_depth.items()):
    print(f'  {depth} tag: {count} token')

if warning_types:
    print(f'\nEn Sik Uyarilar (ilk 10):')
    for w, count in warning_types.most_common(10):
        print(f'  {count:>5}x | {w}')

# Sonuc degerlendirmesi
coverage_pct = 100 * valid_fst / total
clean_pct = 100 * (total - has_warnings) / total
print(f'\n{"=" * 60}')
if coverage_pct > 95 and clean_pct > 70:
    print(f'SONUC: Coverage IIII ({coverage_pct:.1f}%), temiz {clean_pct:.1f}%')
    print(f'Bigram modele gecebilirsin!')
elif coverage_pct > 90:
    print(f'SONUC: Coverage ORTA ({coverage_pct:.1f}%), bazi iyilestirmeler gerekebilir')
else:
    print(f'SONUC: Coverage DUSUK ({coverage_pct:.1f}%), mapping gozden gecirilmeli')


## Disambiguator State Abstraction

FST tam analiz cok sparse:
```
git+VERB+NEG+FUT+1PL+DER:CAUS+DER:PASS  (unique string)
sev+VERB+PRES.CONT+1SG                   (baska unique string)
```

Disambiguator state = **lemma-bagimsiz + DER-siz pattern**:
```
VERB+NEG+FUT+1PL    (ogrenilebilir pattern)
VERB+PRES.CONT+1SG  (ogrenilebilir pattern)
```

Kural:
- FST morphotactics **DEGISMEZ**
- Derivational structure **SADELESSTIRILMEZ**
- Sadece disambiguator icin **state abstraction** yapilir


In [ ]:
# ============================================================
# BOLUM 10: STATE ABSTRACTION KATMANI
# ============================================================
# FST output -> Disambiguator State donusumu
# Kural: Lemma cikar, DER.* cikar, core POS+FEATURE pattern tut

import re

# Cikarilacak tag pattern'lari (derivational + lemma)
DER_PATTERN = re.compile(r'\+DER\.[a-z]+')  # +DER.lik, +DER.ci vb.

def extract_disambiguator_state(fst_analysis):
    # Gecerlilik kontrolu
    if '+' not in fst_analysis:
        return fst_analysis
    
    parts = fst_analysis.split('+')
    
    # parts[0] = lemma, parts[1] = POS, parts[2:] = features
    # Lemma cikar
    tags_only = parts[1:]  # POS + features
    
    # DER.* tag'larini cikar
    filtered = [t for t in tags_only if not t.startswith('DER.')]
    
    # PUNCT ozel durum
    if filtered and filtered[0].startswith('PUNCT'):
        return 'PUNCT'
    
    return '+'.join(filtered) if filtered else 'UNK'


# ── Test ──
test_cases = [
    'sev+VERB+PRES.CONT+1SG',
    'git+VERB+NEG+FUT+1PL',
    'ev+NOUN+PL+ABL',
    'liman+NOUN+ABL',
    'kitap+NOUN+DER.lik',
    'is+NOUN+DER.ci+PL+DAT',
    'bak+VERB+ABIL+NEG+PAST+3SG',
    'guzel+ADJ',
    'sen+PRON+ACC',
    'de+VERB+INFER+3SG',
    '.+PUNCT.period',
]

print('STATE ABSTRACTION ORNEKLERI')
print('=' * 65)
print(f'{"FST Tam Analiz":<35} {"Disambiguator State"}')
print('-' * 65)
for fst in test_cases:
    state = extract_disambiguator_state(fst)
    print(f'{fst:<35} {state}')

print(f'\nDikkat: DER.* taglari state\'den cikarildi.')
print(f'FST output DEGISMEZ, sadece disambiguator bakar.')


In [ ]:
# ============================================================
# BOLUM 11: BIGRAM TRANSITION MODEL EGITIMI
# ============================================================
# UD gold veriden state-level bigram olasiliklari ogrenilir.
# State = POS+FEATURE pattern (lemma ve DER haric)

import math
from collections import Counter, defaultdict

def build_sentence_states(mapped_data, conllu_path):
    # CoNLL-U'dan cumle sinirlarini bul
    sentences = []
    current_sent = []
    
    with open(conllu_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # Cumle sonu
                if current_sent:
                    sentences.append(current_sent)
                    current_sent = []
                continue
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            if len(parts) < 10:
                continue
            if '-' in parts[0] or '.' in parts[0]:
                continue
            current_sent.append(parts[1])  # FORM
        if current_sent:
            sentences.append(current_sent)
    
    return sentences


class BigramTransitionModel:
    def __init__(self, smoothing=0.01):
        self.smoothing = smoothing
        self.bigram_counts = Counter()     # (prev_state, curr_state) -> count
        self.unigram_counts = Counter()    # state -> count
        self.context_counts = Counter()    # prev_state -> toplam gecis
        self.states = set()
    
    def train(self, mapped_data):
        # Cumle sinirlarini bul: PUNCT gorulunce cumle biter
        prev_state = 'START'
        
        for tok in mapped_data:
            fst = tok['fst_analysis']
            curr_state = extract_disambiguator_state(fst)
            
            self.unigram_counts[curr_state] += 1
            self.bigram_counts[(prev_state, curr_state)] += 1
            self.context_counts[prev_state] += 1
            self.states.add(curr_state)
            
            if curr_state == 'PUNCT':
                prev_state = 'START'
            else:
                prev_state = curr_state
        
        self.states.add('START')
    
    def log_prob(self, prev_state, curr_state):
        # Laplace smoothing
        count = self.bigram_counts.get((prev_state, curr_state), 0)
        context = self.context_counts.get(prev_state, 0)
        V = len(self.states)
        prob = (count + self.smoothing) / (context + self.smoothing * V)
        return math.log(prob)
    
    def top_transitions(self, state, n=5):
        # Bir state'den en olasi gecisler
        transitions = []
        for (prev, curr), count in self.bigram_counts.items():
            if prev == state:
                transitions.append((curr, count))
        return sorted(transitions, key=lambda x: -x[1])[:n]
    
    def summary(self):
        total_tokens = sum(self.unigram_counts.values())
        print(f'BIGRAM MODEL OZETI')
        print(f'={"=" * 50}')
        print(f'Toplam token:        {total_tokens}')
        print(f'Unique state sayisi: {len(self.states)}')
        print(f'Unique bigram:       {len(self.bigram_counts)}')
        print(f'Smoothing alpha:     {self.smoothing}')
        
        # Sparsity
        possible = len(self.states) ** 2
        observed = len(self.bigram_counts)
        sparsity = 100 * (1 - observed / possible)
        print(f'Sparsity:            {sparsity:.1f}%')
        
        # En sik state'ler
        print(f'\nEn Sik 15 State:')
        for state, count in self.unigram_counts.most_common(15):
            pct = 100 * count / total_tokens
            bar = '#' * int(pct)
            print(f'  {state:<30} {count:>5} ({pct:>5.1f}%) {bar}')


# ── EGITIM ──
bigram_model = BigramTransitionModel(smoothing=0.01)
bigram_model.train(mapped_data)
bigram_model.summary()


In [ ]:
# ============================================================
# BOLUM 12: BIGRAM GECIS ANALIZI
# ============================================================

# Onemli state'lerden gecis olasiliklarini goster
key_states = [
    'START',
    'NOUN',
    'NOUN+ACC',
    'NOUN+PL+DAT',
    'VERB+PRES.CONT+3SG',
    'VERB+PAST+3SG',
    'VERB+FUT+1PL',
    'ADJ',
    'PRON+ACC',
    'CONJ',
]

print('BIGRAM GECIS ANALIZI')
print('=' * 70)

for state in key_states:
    tops = bigram_model.top_transitions(state, n=5)
    if tops:
        print(f'\n{state} ->')
        for next_state, count in tops:
            log_p = bigram_model.log_prob(state, next_state)
            print(f'  {next_state:<30} count={count:>4}  log_p={log_p:.3f}')
    else:
        print(f'\n{state} -> (veri yok)')

# ── Turkce SOV kontrol ──
print(f'\n{"=" * 70}')
print(f'TURKCE SOV KONTROL:')

# Cumleler genelde VERB/PUNCT ile biter mi?
pre_punct = bigram_model.top_transitions('PUNCT', n=3)
print(f'\nPUNCT sonrasi (cumle basi):')
# Aslinda PUNCT -> START -> X
start_tops = bigram_model.top_transitions('START', n=5)
for s, c in start_tops:
    print(f'  START -> {s:<25} ({c})')

# PUNCT oncesi ne geliyor?
before_punct = Counter()
for (prev, curr), count in bigram_model.bigram_counts.items():
    if curr == 'PUNCT':
        before_punct[prev] += count

print(f'\nPUNCT oncesi (cumle sonu):')
for state, count in before_punct.most_common(8):
    print(f'  {state:<30} -> PUNCT ({count})')


In [ ]:
# ============================================================
# BOLUM 13: MODEL EXPORT + VITERBI ENTEGRASYONU
# ============================================================
# Bigram modelini JSON olarak kaydet.
# Bu dosya FST disambiguator'a yuklenecek.

import json

def export_bigram_model(model, output_path):
    # JSON-serializable format
    export = {
        'metadata': {
            'total_tokens': sum(model.unigram_counts.values()),
            'unique_states': len(model.states),
            'unique_bigrams': len(model.bigram_counts),
            'smoothing': model.smoothing,
        },
        'unigrams': dict(model.unigram_counts),
        'bigrams': {
            f'{prev}|||{curr}': count
            for (prev, curr), count in model.bigram_counts.items()
        },
        'states': sorted(list(model.states)),
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(export, f, ensure_ascii=False, indent=2)
    
    print(f'Bigram model kaydedildi: {output_path}')
    print(f'  States: {len(model.states)}')
    print(f'  Bigrams: {len(model.bigram_counts)}')


def load_bigram_model(json_path):
    # Kaydedilmis modeli geri yukle
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    model = BigramTransitionModel(smoothing=data['metadata']['smoothing'])
    model.unigram_counts = Counter(data['unigrams'])
    model.states = set(data['states'])
    
    for key, count in data['bigrams'].items():
        prev, curr = key.split('|||')
        model.bigram_counts[(prev, curr)] = count
        model.context_counts[prev] += count
    
    return model


# ── Viterbi Disambiguator Onizleme ──
def viterbi_disambiguate(candidates_per_position, bigram_model):
    # candidates_per_position: list of list of (fst_analysis, score)
    # Her pozisyon icin FST candidate'leri ve state'leri
    n = len(candidates_per_position)
    if n == 0:
        return []
    
    # Trellis: best_score[t][candidate_idx] = (log_prob, backpointer)
    best = [{} for _ in range(n)]
    back = [{} for _ in range(n)]
    
    # Init
    for i, (fst, _) in enumerate(candidates_per_position[0]):
        state = extract_disambiguator_state(fst)
        best[0][i] = bigram_model.log_prob('START', state)
    
    # Forward
    for t in range(1, n):
        for i, (curr_fst, _) in enumerate(candidates_per_position[t]):
            curr_state = extract_disambiguator_state(curr_fst)
            max_score = float('-inf')
            best_prev = 0
            
            for j, (prev_fst, _) in enumerate(candidates_per_position[t-1]):
                prev_state = extract_disambiguator_state(prev_fst)
                score = best[t-1].get(j, -100) + bigram_model.log_prob(prev_state, curr_state)
                if score > max_score:
                    max_score = score
                    best_prev = j
            
            best[t][i] = max_score
            back[t][i] = best_prev
    
    # Backtrack
    best_last = max(best[n-1], key=best[n-1].get)
    path = []
    idx = best_last
    for t in range(n-1, -1, -1):
        path.append(candidates_per_position[t][idx][0])
        if t > 0:
            idx = back[t][idx]
    
    return list(reversed(path))


# ── MODEL KAYDET ──
export_bigram_model(bigram_model, 'bigram_transition_model.json')

# ── ORNEK VITERBI CALISTIRMA ──
# Simule edilmis candidate listesi
example_candidates = [
    # Pozisyon 0: "cocuk"
    [('cocuk+NOUN', 1.0), ('cocuk+ADJ', 0.5)],
    # Pozisyon 1: "okula"
    [('okul+NOUN+DAT', 1.0)],
    # Pozisyon 2: "gitti"
    [('git+VERB+PAST+3SG', 1.0), ('git+NOUN+POSS.3SG', 0.3)],
    # Pozisyon 3: "."
    [('.+PUNCT.period', 1.0)],
]

result = viterbi_disambiguate(example_candidates, bigram_model)
print(f'\nVITERBI ORNEK:')
print(f'Girdi:  cocuk okula gitti .')
print(f'Secim:  {" | ".join(result)}')
print(f'\nEntegrasyon hazir. Bu model FST disambiguator\'a yuklenebilir.')


## Interpolated Bigram HMM Disambiguator

Mimari:
```
score = transition_log_prob(prev_state, curr_state)
      + emission_log_prob(surface, curr_state)
      + unigram_log_prob(curr_state)
```

Transition interpolasyonu:
```
P_trans = lambda * P_bigram(state|prev) + (1-lambda) * P_unigram(state)
```

Kurallar:
- FST morphotactics **DEGISMEZ**
- PUNCT tek candidate ise **direkt sec**, transition'a katma
- State = POS+FEATURE pattern (lemma+DER haric)


In [ ]:
# ============================================================
# BOLUM 14: INTERPOLATED BIGRAM HMM DISAMBIGUATOR
# ============================================================

import math
import json
import re
from collections import Counter, defaultdict

class InterpolatedHMMDisambiguator:
    
    def __init__(self, lam=0.7, smoothing=0.01):
        # lam: bigram interpolasyon agirligi
        # P = lam * P_bigram + (1-lam) * P_unigram
        self.lam = lam
        self.smoothing = smoothing
        
        # Transition counts
        self.bigram_counts = Counter()    # (prev_state, curr_state)
        self.context_counts = Counter()   # prev_state -> toplam
        self.unigram_counts = Counter()   # state -> count
        self.total_tokens = 0
        
        # Emission counts: P(state | surface)
        self.emission_counts = defaultdict(Counter)  # surface -> {state: count}
        self.surface_totals = Counter()              # surface -> toplam
        
        self.states = set()
    
    # ── TRAINING ──
    
    def train(self, mapped_data):
        prev_state = 'START'
        
        for tok in mapped_data:
            fst = tok['fst_analysis']
            surface = tok['form'].lower()
            curr_state = extract_disambiguator_state(fst)
            
            # PUNCT bypass: sayma ama transition zincirini kirma
            if curr_state == 'PUNCT':
                prev_state = 'START'  # Cumle sonu -> yeni cumle
                continue
            
            # Transition
            self.bigram_counts[(prev_state, curr_state)] += 1
            self.context_counts[prev_state] += 1
            self.unigram_counts[curr_state] += 1
            self.total_tokens += 1
            
            # Emission: P(state | surface)
            self.emission_counts[surface][curr_state] += 1
            self.surface_totals[surface] += 1
            
            self.states.add(curr_state)
            prev_state = curr_state
        
        self.states.add('START')
    
    # ── PROBABILITIES ──
    
    def _p_unigram(self, state):
        # Laplace smoothed unigram
        V = len(self.states)
        count = self.unigram_counts.get(state, 0)
        return (count + self.smoothing) / (self.total_tokens + self.smoothing * V)
    
    def _p_bigram(self, prev_state, curr_state):
        # Laplace smoothed bigram
        V = len(self.states)
        count = self.bigram_counts.get((prev_state, curr_state), 0)
        context = self.context_counts.get(prev_state, 0)
        return (count + self.smoothing) / (context + self.smoothing * V)
    
    def transition_log_prob(self, prev_state, curr_state):
        # Interpolated: P = lam * P_bi + (1-lam) * P_uni
        p_bi = self._p_bigram(prev_state, curr_state)
        p_uni = self._p_unigram(curr_state)
        p_interp = self.lam * p_bi + (1 - self.lam) * p_uni
        return math.log(max(p_interp, 1e-15))
    
    def emission_log_prob(self, surface, state):
        # P(state | surface) - surface-conditioned prior
        surface = surface.lower()
        if surface not in self.emission_counts:
            # Gorulmemis surface -> uniform
            return 0.0
        
        count = self.emission_counts[surface].get(state, 0)
        total = self.surface_totals[surface]
        V = len(self.states)
        p = (count + self.smoothing) / (total + self.smoothing * V)
        return math.log(max(p, 1e-15))
    
    # ── VITERBI ──
    
    def disambiguate(self, candidate_lists):
        # candidate_lists: list of list of fst_analysis strings
        #   [["ev+NOUN", "ev+NOUN+DAT"], ["git+VERB+PAST+3SG"], ...]
        
        n = len(candidate_lists)
        if n == 0:
            return []
        
        # PUNCT bypass: tek candidate ve PUNCT ise direkt sec
        # Pre-process: PUNCT pozisyonlarini isaretle
        punct_positions = set()
        for t, cands in enumerate(candidate_lists):
            if len(cands) == 1 and 'PUNCT' in cands[0]:
                punct_positions.add(t)
        
        # Non-PUNCT pozisyonlari icin Viterbi
        # Trellis: best[t][i] = (score, backpointer)
        best = [{} for _ in range(n)]
        back = [{} for _ in range(n)]
        
        # ── Init (t=0) ──
        if 0 in punct_positions:
            best[0][0] = 0.0
        else:
            for i, fst in enumerate(candidate_lists[0]):
                state = extract_disambiguator_state(fst)
                surface = fst.split('+')[0] if '+' in fst else fst
                score = (self.transition_log_prob('START', state)
                         + self.emission_log_prob(surface, state))
                best[0][i] = score
        
        # ── Forward ──
        for t in range(1, n):
            if t in punct_positions:
                # PUNCT: direkt sec, onceki en iyi state'den devam
                best_prev_idx = max(best[t-1], key=best[t-1].get) if best[t-1] else 0
                best[t][0] = best[t-1].get(best_prev_idx, 0.0)
                back[t][0] = best_prev_idx
                continue
            
            # Onceki state'i bul (PUNCT ise START olarak kabul et)
            prev_is_punct = (t-1) in punct_positions
            
            for i, curr_fst in enumerate(candidate_lists[t]):
                curr_state = extract_disambiguator_state(curr_fst)
                curr_surface = curr_fst.split('+')[0] if '+' in curr_fst else curr_fst
                
                max_score = float('-inf')
                best_prev = 0
                
                if prev_is_punct:
                    # Onceki PUNCT idi -> prev_state = START
                    score = (best[t-1].get(0, 0.0)
                             + self.transition_log_prob('START', curr_state)
                             + self.emission_log_prob(curr_surface, curr_state))
                    max_score = score
                    best_prev = 0
                else:
                    for j, prev_fst in enumerate(candidate_lists[t-1]):
                        prev_state = extract_disambiguator_state(prev_fst)
                        
                        score = (best[t-1].get(j, -100)
                                 + self.transition_log_prob(prev_state, curr_state)
                                 + self.emission_log_prob(curr_surface, curr_state))
                        
                        if score > max_score:
                            max_score = score
                            best_prev = j
                
                best[t][i] = max_score
                back[t][i] = best_prev
        
        # ── Backtrack ──
        if not best[n-1]:
            return [cands[0] if cands else '' for cands in candidate_lists]
        
        best_last = max(best[n-1], key=best[n-1].get)
        path = []
        idx = best_last
        for t in range(n-1, -1, -1):
            path.append(candidate_lists[t][idx])
            if t > 0:
                idx = back[t].get(idx, 0)
        
        return list(reversed(path))
    
    # ── SAVE / LOAD ──
    
    def save(self, path):
        data = {
            'metadata': {
                'lambda': self.lam,
                'smoothing': self.smoothing,
                'total_tokens': self.total_tokens,
                'unique_states': len(self.states),
                'unique_bigrams': len(self.bigram_counts),
                'unique_surfaces': len(self.emission_counts),
            },
            'unigrams': dict(self.unigram_counts),
            'bigrams': {
                f'{p}|||{c}': cnt
                for (p, c), cnt in self.bigram_counts.items()
            },
            'emissions': {
                surf: dict(state_counts)
                for surf, state_counts in self.emission_counts.items()
            },
            'states': sorted(list(self.states)),
        }
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f'Model kaydedildi: {path}')
    
    @classmethod
    def load(cls, path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        m = cls(lam=data['metadata']['lambda'],
                smoothing=data['metadata']['smoothing'])
        m.total_tokens = data['metadata']['total_tokens']
        m.unigram_counts = Counter(data['unigrams'])
        m.states = set(data['states'])
        
        for key, cnt in data['bigrams'].items():
            prev, curr = key.split('|||')
            m.bigram_counts[(prev, curr)] = cnt
            m.context_counts[prev] += cnt
        
        for surf, state_dict in data['emissions'].items():
            for state, cnt in state_dict.items():
                m.emission_counts[surf][state] = cnt
                m.surface_totals[surf] += cnt
        
        return m
    
    # ── SUMMARY ──
    
    def summary(self):
        print(f'INTERPOLATED HMM DISAMBIGUATOR')
        print(f'={"=" * 50}')
        print(f'Lambda (bigram weight):  {self.lam}')
        print(f'Smoothing alpha:         {self.smoothing}')
        print(f'Toplam token (no PUNCT): {self.total_tokens}')
        print(f'Unique state:            {len(self.states)}')
        print(f'Unique bigram:           {len(self.bigram_counts)}')
        print(f'Unique surface:          {len(self.emission_counts)}')
        
        possible = len(self.states) ** 2
        observed = len(self.bigram_counts)
        sparsity = 100 * (1 - observed / max(possible, 1))
        print(f'Bigram sparsity:         {sparsity:.1f}%')
        
        print(f'\nEn Sik 10 State:')
        for s, c in self.unigram_counts.most_common(10):
            print(f'  {s:<30} {c:>5}')


# ── EGITIM ──
hmm = InterpolatedHMMDisambiguator(lam=0.7, smoothing=0.01)
hmm.train(mapped_data)
hmm.summary()
hmm.save('hmm_disambiguator_model.json')


In [ ]:
# ============================================================
# BOLUM 15: ENTEGRE VITERBI TESTI
# ============================================================
# FST analyzer olmadan simule edilmis candidate'lerle test.
# Gercek FST icin Colab'da pynini gerekir.

# ── Simule edilmis cumleler ──
# Her pozisyon: FST'nin uretecegi candidate listesi

test_sentences = [
    {
        'text': 'cocuk okula gitti',
        'candidates': [
            ['cocuk+NOUN', 'cocuk+ADJ'],
            ['okul+NOUN+DAT'],
            ['git+VERB+PAST+3SG', 'git+NOUN+POSS.3SG'],
        ]
    },
    {
        'text': 'ben seni seviyorum',
        'candidates': [
            ['ben+PRON'],
            ['sen+PRON+ACC', 'sen+NOUN+ACC'],
            ['sev+VERB+PRES.CONT+1SG'],
        ]
    },
    {
        'text': 'guzel ev almak istiyorum',
        'candidates': [
            ['guzel+ADJ', 'guzel+NOUN'],
            ['ev+NOUN', 'ev+NOUN+ACC'],
            ['al+VERB+VNOUN', 'al+NOUN+DAT'],
            ['iste+VERB+PRES.CONT+1SG'],
        ]
    },
    {
        'text': 'yarin okula gelmeyecek',
        'candidates': [
            ['yarin+NOUN', 'yarin+ADV'],
            ['okul+NOUN+DAT'],
            ['gel+VERB+NEG+FUT+3SG', 'gel+NOUN+PL+DAT'],
        ]
    },
    {
        'text': 'kitabi masa uzerine koydum',
        'candidates': [
            ['kitap+NOUN+ACC', 'kitap+NOUN+POSS.3SG'],
            ['masa+NOUN', 'masa+ADJ'],
            ['uzeri+NOUN+DAT', 'uzer+NOUN+POSS.3SG+DAT'],
            ['koy+VERB+PAST+1SG'],
        ]
    },
]

print('VITERBI DISAMBIGUASYON SONUCLARI')
print('=' * 70)

for sent in test_sentences:
    result = hmm.disambiguate(sent['candidates'])
    
    print(f'\nCumle: {sent["text"]}')
    words = sent['text'].split()
    for i, (word, selected) in enumerate(zip(words, result)):
        state = extract_disambiguator_state(selected)
        n_cands = len(sent['candidates'][i])
        marker = '*' if n_cands > 1 else ' '
        print(f'  {marker} {word:<15} -> {selected:<30} [state: {state}] ({n_cands} cand)')

print(f'\n* = Birden fazla candidate arasinda secim yapildi')


## FST Motor Entegrasyonu

Asagidaki kod `turkish_morphological_segmentation.ipynb`'deki
`ContextAwareDisambiguator` ile birlestirilecek.

Colab'da kullanim:
```python
# 1. FST analyzer ve HMM modelini yukle
analyzer = build_analyzer(lexicon)
hmm = InterpolatedHMMDisambiguator.load('hmm_disambiguator_model.json')

# 2. Cumleyi analiz et
result = analyze_with_hmm(sentence, analyzer, hmm)
```


In [ ]:
# ============================================================
# BOLUM 16: FST + HMM ENTEGRASYON FONKSIYONU
# ============================================================
# Bu fonksiyon pynini FST analyzer ile HMM disambiguator'u birlestirir.
# Colab'da calistirilmak uzere hazirlanmistir.

import re

def tokenize_turkish(text):
    return re.findall(r"[\w']+|[.,!?;:]", text)


def analyze_with_hmm(sentence, fst_analyzer, hmm_model, analyze_func=None):
    # analyze_func: word -> list[str]  (FST candidate listesi)
    # Eger verilmezse pynini compose kullanilir
    
    tokens = tokenize_turkish(sentence.lower())
    
    candidate_lists = []
    for word in tokens:
        if analyze_func:
            candidates = analyze_func(word)
        else:
            # Fallback: pynini varsa
            try:
                import pynini
                lattice = pynini.compose(word, fst_analyzer)
                candidates = []
                for path in lattice.paths().ostrings():
                    if path not in candidates:
                        candidates.append(path)
            except Exception:
                candidates = []
        
        # Candidate yoksa OOV
        if not candidates or (len(candidates)==1 and 'No analysis' in candidates[0]):
            candidates = [f'{word}+UNK+OOV']
        
        candidate_lists.append(candidates)
    
    # HMM Viterbi
    best_path = hmm_model.disambiguate(candidate_lists)
    
    # Sonuclari formatla
    results = []
    for word, selected in zip(tokens, best_path):
        state = extract_disambiguator_state(selected)
        results.append({
            'token': word,
            'analysis': selected,
            'state': state,
        })
    
    return results


# ── Colab'da kullanim ornegi (yorum satiri) ──
# from turkish_morphological_segmentation import *
# lex = normalize_lexicon(load_lexicon('MASTER_FINAL_ROOTS.json'))
# analyzer = build_analyzer(lex)
# hmm = InterpolatedHMMDisambiguator.load('hmm_disambiguator_model.json')
# 
# result = analyze_with_hmm(
#     'cocuk okula gitti',
#     analyzer, hmm,
#     analyze_func=lambda w: analyze_word(w, analyzer)
# )
# for r in result:
#     print(f'{r["token"]:<15} {r["analysis"]:<30} [{r["state"]}]')

print('FST + HMM entegrasyon fonksiyonu hazir.')
print('Colab\'da pynini ile calistirmak icin yukaridaki yorumlari acin.')


In [ ]:
# ============================================================
# BOLUM 17: LAMBDA TUNING (Opsiyonel)
# ============================================================
# Farkli lambda degerleri ile interpolasyon etkisini test et.

import math

def test_lambda_values(mapped_data, lambdas=[0.5, 0.6, 0.7, 0.8, 0.9]):
    print('LAMBDA TUNING')
    print(f'={"=" * 50}')
    
    for lam in lambdas:
        model = InterpolatedHMMDisambiguator(lam=lam, smoothing=0.01)
        model.train(mapped_data)
        
        # Perplexity hesapla (training set uzerinde)
        total_log_prob = 0.0
        n = 0
        prev_state = 'START'
        
        for tok in mapped_data:
            fst = tok['fst_analysis']
            curr_state = extract_disambiguator_state(fst)
            if curr_state == 'PUNCT':
                prev_state = 'START'
                continue
            
            surface = tok['form'].lower()
            log_p = (model.transition_log_prob(prev_state, curr_state)
                     + model.emission_log_prob(surface, curr_state))
            total_log_prob += log_p
            n += 1
            prev_state = curr_state
        
        avg_log_p = total_log_prob / max(n, 1)
        perplexity = math.exp(-avg_log_p)
        
        bar = '#' * int(50 / max(perplexity, 0.1))
        print(f'  lambda={lam:.1f}  avg_logp={avg_log_p:.3f}  ppl={perplexity:.1f}  {bar}')

test_lambda_values(mapped_data)
print(f'\nDusuk perplexity = daha iyi model.')
print(f'En iyi lambda degerini secip modeli yeniden egitin.')


## Model Dogrulama Testleri

3 kritik test:
1. **Satir toplami**: Her prev_state icin transition olasiliklari ~1'e toplanmali
2. **Unigram kontrolu**: Viterbi recursion'da unigram olmali
3. **Deterministik oyuncak veri**: START->A 100, START->B 1 ise her zaman A secilmeli


In [ ]:
# ============================================================
# TEST 1: SATIR TOPLAMI (Row Sum Normalization)
# ============================================================
# Her prev_state icin tum gecis olasiliklari toplamda ~1 olmali.

import math

print('TEST 1: SATIR TOPLAMI')
print('=' * 60)

# HMM modeldeki tum prev_state'ler
all_prev_states = set()
for (prev, curr) in hmm.bigram_counts:
    all_prev_states.add(prev)

pass_count = 0
fail_count = 0
results = []

for prev in sorted(all_prev_states):
    total_prob = 0.0
    for state in hmm.states:
        # Interpolated probability
        p = hmm.lam * hmm._p_bigram(prev, state) + (1 - hmm.lam) * hmm._p_unigram(state)
        total_prob += p
    
    ok = abs(total_prob - 1.0) < 0.05
    if ok:
        pass_count += 1
    else:
        fail_count += 1
    results.append((prev, total_prob, ok))

# Ilk 10 ve sorunlulari goster
for prev, total, ok in results[:10]:
    status = 'OK' if ok else 'FAIL'
    print(f'  {prev:<30} sum={total:.6f}  [{status}]')

if fail_count > 0:
    print(f'\nFAIL olan state\'ler:')
    for prev, total, ok in results:
        if not ok:
            print(f'  {prev:<30} sum={total:.6f}')

print(f'\nSONUC: {pass_count} PASS, {fail_count} FAIL / {len(results)} state')
if fail_count == 0:
    print('NORMALIZATION DOGRU!')


In [ ]:
# ============================================================
# TEST 2: UNIGRAM + EMISSION VARLIK KONTROLU
# ============================================================
# Viterbi recursion satirinda 3 bilesen olmali.

import inspect

print('TEST 2: VITERBI RECURSION BILESEN KONTROLU')
print('=' * 60)

# Kaynak kodunu oku
source = inspect.getsource(hmm.disambiguate)

checks = [
    ('transition_log_prob', 'Interpolated Bigram (lam*P_bi + (1-lam)*P_uni)'),
    ('emission_log_prob', 'Surface-conditioned emission P(state|surface)'),
]

all_pass = True
for func_name, desc in checks:
    found = func_name in source
    status = 'OK' if found else 'EKSIK!'
    if not found:
        all_pass = False
    print(f'  [{status}] {desc}')
    print(f'         -> {func_name}() fonksiyonu {"MEVCUT" if found else "YOK"}')

# Unigram, transition_log_prob icinde mi?
trans_source = inspect.getsource(hmm.transition_log_prob)
has_interp = 'lam' in trans_source and '_p_unigram' in trans_source and '_p_bigram' in trans_source
status = 'OK' if has_interp else 'EKSIK!'
if not has_interp:
    all_pass = False
print(f'  [{status}] Interpolation (lam * P_bi + (1-lam) * P_uni)')
print(f'         -> transition_log_prob icinde {"VAR" if has_interp else "YOK"}')

# PUNCT bypass
train_source = inspect.getsource(hmm.train)
has_punct_bypass = 'PUNCT' in train_source and 'continue' in train_source
status = 'OK' if has_punct_bypass else 'EKSIK!'
if not has_punct_bypass:
    all_pass = False
print(f'  [{status}] PUNCT bypass (transition zincirinden dislanmis)')

print(f'\nSONUC: {"TUM KONTROLLER GECTI" if all_pass else "EKSIKLER VAR!"}')

# Gercek recursion satirini goster
print(f'\nViterbi recursion satiri (kaynak koddan):')
for line in source.split('\n'):
    if 'transition_log_prob' in line or 'emission_log_prob' in line:
        print(f'  >>> {line.strip()}')


In [ ]:
# ============================================================
# TEST 3: DETERMINISTIK OYUNCAK VERI
# ============================================================
# START -> A: 100 kez, START -> B: 1 kez
# Cumle: w1 (candidate: A veya B)
# Model HER ZAMAN A secmeli.

print('TEST 3: DETERMINISTIK OYUNCAK VERI')
print('=' * 60)

# Oyuncak model olustur
toy = InterpolatedHMMDisambiguator(lam=0.7, smoothing=0.001)

# Manuel training: START->A 100 kez, START->B 1 kez
toy.states = {'START', 'A', 'B'}
toy.bigram_counts[('START', 'A')] = 100
toy.bigram_counts[('START', 'B')] = 1
toy.context_counts['START'] = 101
toy.unigram_counts['A'] = 100
toy.unigram_counts['B'] = 1
toy.total_tokens = 101

# Olasililklari kontrol et
p_a = toy.lam * toy._p_bigram('START', 'A') + (1 - toy.lam) * toy._p_unigram('A')
p_b = toy.lam * toy._p_bigram('START', 'B') + (1 - toy.lam) * toy._p_unigram('B')
print(f'  P(A|START) = {p_a:.6f}')
print(f'  P(B|START) = {p_b:.6f}')
print(f'  P(A) / P(B) = {p_a/p_b:.1f}x')

# Viterbi ile test: candidate = [A, B]
result = toy.disambiguate([['w1+A', 'w1+B']])
selected_state = extract_disambiguator_state(result[0])

print(f'\n  Viterbi secimi: {result[0]}')
print(f'  Secilen state:  {selected_state}')

if selected_state == 'A':
    print(f'  SONUC: PASS - Model dogru sekilde A secti!')
else:
    print(f'  SONUC: FAIL - Model yanlis secim yapti!')

# Ek test: 3 pozisyonlu cumle
# A->A: 80, A->B: 20, B->A: 50, B->B: 50
toy.bigram_counts[('A', 'A')] = 80
toy.bigram_counts[('A', 'B')] = 20
toy.bigram_counts[('B', 'A')] = 50
toy.bigram_counts[('B', 'B')] = 50
toy.context_counts['A'] = 100
toy.context_counts['B'] = 100

result2 = toy.disambiguate([
    ['w1+A', 'w1+B'],  # pos 0
    ['w2+A', 'w2+B'],  # pos 1
    ['w3+A', 'w3+B'],  # pos 2
])

print(f'\n  3-pozisyon testi:')
print(f'  Beklenen: A -> A -> A (cunku START->A ve A->A baskın)')
states = [extract_disambiguator_state(r) for r in result2]
print(f'  Sonuc:    {" -> ".join(states)}')
all_a = all(s == 'A' for s in states)
print(f'  SONUC: {"PASS" if all_a else "FAIL"}')

print(f'\n{"=" * 60}')
print(f'TUM TESTLER TAMAMLANDI.')


## DUZELTME: Temiz Viterbi Recursion

Doğru formül:
```
t=0:  score = trans(START, s)  + emit(w, s)
t>0:  score = prev_score      + trans(prev_s, s) + emit(w, s)
```
Başka hiçbir şey eklenmeyecek.


In [ ]:
# ============================================================
# DUZELTILMIS VITERBI: MINIMAL, MATEMATIKSEL DOGRU
# ============================================================

import math
import json
from collections import Counter, defaultdict

class CleanHMMDisambiguator:
    
    def __init__(self, lam=0.7, smoothing=0.01):
        self.lam = lam
        self.smoothing = smoothing
        self.bigram_counts = Counter()
        self.context_counts = Counter()
        self.unigram_counts = Counter()
        self.total_tokens = 0
        self.emission_counts = defaultdict(Counter)
        self.surface_totals = Counter()
        self.states = set()
    
    # ── TRAINING ──
    def train(self, mapped_data):
        prev_state = 'START'
        for tok in mapped_data:
            fst = tok['fst_analysis']
            surface = tok['form'].lower()
            curr_state = extract_disambiguator_state(fst)
            
            # PUNCT: transition'a KATMA, cumle siniri olarak isle
            if curr_state == 'PUNCT':
                prev_state = 'START'
                continue
            
            self.bigram_counts[(prev_state, curr_state)] += 1
            self.context_counts[prev_state] += 1
            self.unigram_counts[curr_state] += 1
            self.total_tokens += 1
            self.emission_counts[surface][curr_state] += 1
            self.surface_totals[surface] += 1
            self.states.add(curr_state)
            prev_state = curr_state
        
        self.states.add('START')
    
    # ── PROBABILITIES ──
    def _p_uni(self, state):
        V = len(self.states)
        c = self.unigram_counts.get(state, 0)
        return (c + self.smoothing) / (self.total_tokens + self.smoothing * V)
    
    def _p_bi(self, prev, curr):
        V = len(self.states)
        c = self.bigram_counts.get((prev, curr), 0)
        ctx = self.context_counts.get(prev, 0)
        return (c + self.smoothing) / (ctx + self.smoothing * V)
    
    def trans(self, prev, curr):
        # Interpolated transition: lam * P_bi + (1-lam) * P_uni
        p = self.lam * self._p_bi(prev, curr) + (1 - self.lam) * self._p_uni(curr)
        return math.log(max(p, 1e-15))
    
    def emit(self, surface, state):
        # P(state | surface)
        surface = surface.lower()
        if surface not in self.emission_counts:
            return 0.0  # Gorulmemis surface -> uniform (log(1)=0)
        V = len(self.states)
        c = self.emission_counts[surface].get(state, 0)
        t = self.surface_totals[surface]
        return math.log(max((c + self.smoothing) / (t + self.smoothing * V), 1e-15))
    
    # ── VITERBI: MINIMAL TEMIZ ──
    def disambiguate(self, surfaces, candidate_lists):
        # surfaces: ['cocuk', 'okula', 'gitti', '.']
        # candidate_lists: [['cocuk+NOUN','cocuk+ADJ'], ['okul+NOUN+DAT'], ...]
        #
        # FORMUL:
        #   t=0:  score[i] = trans(START, s_i)     + emit(w_0, s_i)
        #   t>0:  score[i] = best[t-1][j]          + trans(s_j, s_i) + emit(w_t, s_i)
        #
        # BASKA HICBIR SEY EKLENMEYECEK.
        
        n = len(candidate_lists)
        if n == 0:
            return []
        
        best = [{} for _ in range(n)]   # best[t][i] = log_prob
        back = [{} for _ in range(n)]   # back[t][i] = j (onceki index)
        
        # ── t=0: INITIALIZATION ──
        for i, fst in enumerate(candidate_lists[0]):
            s = extract_disambiguator_state(fst)
            # PUNCT tek candidate ise direkt sec
            if s == 'PUNCT':
                best[0][i] = 0.0
            else:
                best[0][i] = self.trans('START', s) + self.emit(surfaces[0], s)
        
        # ── t>0: RECURSION ──
        for t in range(1, n):
            for i, curr_fst in enumerate(candidate_lists[t]):
                curr_s = extract_disambiguator_state(curr_fst)
                
                # PUNCT: direkt sec, skor olarak onceki en iyiyi al
                if curr_s == 'PUNCT':
                    best_j = max(best[t-1], key=best[t-1].get) if best[t-1] else 0
                    best[t][i] = best[t-1].get(best_j, 0.0)
                    back[t][i] = best_j
                    continue
                
                max_score = float('-inf')
                best_j = 0
                
                for j, prev_fst in enumerate(candidate_lists[t-1]):
                    prev_s = extract_disambiguator_state(prev_fst)
                    
                    # Onceki PUNCT ise -> prev_s = START
                    if prev_s == 'PUNCT':
                        prev_s = 'START'
                    
                    # === RECURSION: TAM BU, BASKA HICBIR SEY ===
                    score = best[t-1][j] + self.trans(prev_s, curr_s) + self.emit(surfaces[t], curr_s)
                    # ================================================
                    
                    if score > max_score:
                        max_score = score
                        best_j = j
                
                best[t][i] = max_score
                back[t][i] = best_j
        
        # ── BACKTRACK ──
        if not best[n-1]:
            return [c[0] for c in candidate_lists]
        
        idx = max(best[n-1], key=best[n-1].get)
        path = []
        for t in range(n-1, -1, -1):
            path.append(candidate_lists[t][idx])
            idx = back[t].get(idx, 0) if t > 0 else 0
        
        return list(reversed(path))
    
    # ── SAVE / LOAD ──
    def save(self, path):
        data = {
            'metadata': {'lambda': self.lam, 'smoothing': self.smoothing,
                         'total_tokens': self.total_tokens,
                         'states': len(self.states), 'bigrams': len(self.bigram_counts),
                         'surfaces': len(self.emission_counts)},
            'unigrams': dict(self.unigram_counts),
            'bigrams': {f'{p}|||{c}': n for (p,c),n in self.bigram_counts.items()},
            'emissions': {s: dict(d) for s,d in self.emission_counts.items()},
            'states': sorted(list(self.states)),
        }
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f'Model kaydedildi: {path}')
    
    @classmethod
    def load(cls, path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        m = cls(lam=data['metadata']['lambda'], smoothing=data['metadata']['smoothing'])
        m.total_tokens = data['metadata']['total_tokens']
        m.unigram_counts = Counter(data['unigrams'])
        m.states = set(data['states'])
        for key, cnt in data['bigrams'].items():
            p, c = key.split('|||')
            m.bigram_counts[(p, c)] = cnt
            m.context_counts[p] += cnt
        for surf, sd in data['emissions'].items():
            for st, cnt in sd.items():
                m.emission_counts[surf][st] = cnt
                m.surface_totals[surf] += cnt
        return m


# ── EGITIM ──
clean_hmm = CleanHMMDisambiguator(lam=0.7, smoothing=0.01)
clean_hmm.train(mapped_data)

print(f'CleanHMMDisambiguator')
print(f'  Tokens: {clean_hmm.total_tokens}')
print(f'  States: {len(clean_hmm.states)}')
print(f'  Bigrams: {len(clean_hmm.bigram_counts)}')
print(f'  Surfaces: {len(clean_hmm.emission_counts)}')
clean_hmm.save('clean_hmm_model.json')


In [ ]:
# ============================================================
# DUZELTILMIS VITERBI: DOGRULAMA TESTLERI
# ============================================================

import inspect

print('RECURSION YAPISI KONTROLU')
print('=' * 60)

# Kaynak koddan recursion satirini bul
src = inspect.getsource(clean_hmm.disambiguate)
lines = src.split('\n')

# trans ve emit iceren satirlari say
trans_lines = [l.strip() for l in lines if 'self.trans(' in l and not l.strip().startswith('#')]
emit_lines = [l.strip() for l in lines if 'self.emit(' in l and not l.strip().startswith('#')]

print(f'trans() cagri sayisi: {len(trans_lines)}')
for i, l in enumerate(trans_lines):
    print(f'  [{i+1}] {l}')

print(f'\nemit() cagri sayisi: {len(emit_lines)}')
for i, l in enumerate(emit_lines):
    print(f'  [{i+1}] {l}')

# Beklenen: 2 trans (init + recursion), 2 emit (init + recursion)
# Recursion icinde SADECE 1 trans + 1 emit olmali
print(f'\nBeklenen: 2 trans (t=0 ve t>0), 2 emit (t=0 ve t>0)')
ok = len(trans_lines) == 2 and len(emit_lines) == 2
print(f'SONUC: {"PASS" if ok else "KONTROL ET"}')

# Ana recursion satirini goster
print(f'\n--- ANA RECURSION SATIRI ---')
for l in lines:
    if 'RECURSION: TAM BU' in l or ('best[t-1][j]' in l and 'trans' in l):
        print(f'>>> {l.strip()}')

# ── DETERMINISTIK OYUNCAK TEST ──
print(f'\n\nOYUNCAK VERI TESTI')
print('=' * 60)

toy = CleanHMMDisambiguator(lam=0.7, smoothing=0.001)
toy.states = {'START', 'NOUN', 'VERB', 'ADJ'}
toy.bigram_counts[('START', 'NOUN')] = 100
toy.bigram_counts[('START', 'ADJ')] = 10
toy.bigram_counts[('START', 'VERB')] = 1
toy.bigram_counts[('NOUN', 'VERB')] = 80
toy.bigram_counts[('NOUN', 'NOUN')] = 20
toy.bigram_counts[('ADJ', 'NOUN')] = 90
toy.bigram_counts[('ADJ', 'VERB')] = 10
toy.context_counts = Counter({p: sum(c for (pp,_),c in toy.bigram_counts.items() if pp==p) for p in ['START','NOUN','ADJ']})
toy.unigram_counts = Counter({'NOUN': 210, 'VERB': 91, 'ADJ': 10})
toy.total_tokens = 311

# Test 1: "cocuk gitti" -> NOUN VERB bekleniyor
r1 = toy.disambiguate(
    ['cocuk', 'gitti'],
    [['cocuk+NOUN', 'cocuk+ADJ'], ['git+VERB', 'git+NOUN']]
)
s1 = [extract_disambiguator_state(x) for x in r1]
print(f'  "cocuk gitti"  -> {s1}  beklenen: [NOUN, VERB]  {"PASS" if s1==["NOUN","VERB"] else "FAIL"}')

# Test 2: "guzel ev" -> ADJ NOUN bekleniyor
r2 = toy.disambiguate(
    ['guzel', 'ev'],
    [['guzel+ADJ', 'guzel+NOUN'], ['ev+NOUN', 'ev+VERB']]
)
s2 = [extract_disambiguator_state(x) for x in r2]
print(f'  "guzel ev"     -> {s2}  beklenen: [ADJ, NOUN]   {"PASS" if s2==["ADJ","NOUN"] else "KONTROL"}')

# Test 3: Tek candidate -> direkt sec
r3 = toy.disambiguate(
    ['okul'],
    [['okul+NOUN']]
)
s3 = [extract_disambiguator_state(x) for x in r3]
print(f'  "okul" (tek)   -> {s3}  beklenen: [NOUN]        {"PASS" if s3==["NOUN"] else "FAIL"}')


## Hyperparameter Sweep: Lambda + Emission Weight

Model tipi: **Interpolated Bigram HMM with conditional lexical emission**

Ayar parametreleri:
- **lambda**: Bigram vs unigram agirligi (0=sadece unigram, 1=sadece bigram)
- **beta**: Emission agirligi (`score = prev + trans + beta * emit`)
- Dusuk perplexity / yuksek accuracy = daha iyi


In [ ]:
# ============================================================
# BOLUM 18: LAMBDA + BETA SWEEP
# ============================================================

import math
import json
from collections import Counter, defaultdict

class TunableHMMDisambiguator:
    # CleanHMM + beta (emission weight) destegi
    
    def __init__(self, lam=0.7, beta=1.0, smoothing=0.01):
        self.lam = lam
        self.beta = beta
        self.smoothing = smoothing
        self.bigram_counts = Counter()
        self.context_counts = Counter()
        self.unigram_counts = Counter()
        self.total_tokens = 0
        self.emission_counts = defaultdict(Counter)
        self.surface_totals = Counter()
        self.states = set()
    
    def train(self, mapped_data):
        prev_state = 'START'
        for tok in mapped_data:
            fst = tok['fst_analysis']
            surface = tok['form'].lower()
            curr_state = extract_disambiguator_state(fst)
            if curr_state == 'PUNCT':
                prev_state = 'START'
                continue
            self.bigram_counts[(prev_state, curr_state)] += 1
            self.context_counts[prev_state] += 1
            self.unigram_counts[curr_state] += 1
            self.total_tokens += 1
            self.emission_counts[surface][curr_state] += 1
            self.surface_totals[surface] += 1
            self.states.add(curr_state)
            prev_state = curr_state
        self.states.add('START')
    
    def _p_uni(self, state):
        V = len(self.states)
        c = self.unigram_counts.get(state, 0)
        return (c + self.smoothing) / (self.total_tokens + self.smoothing * V)
    
    def _p_bi(self, prev, curr):
        V = len(self.states)
        c = self.bigram_counts.get((prev, curr), 0)
        ctx = self.context_counts.get(prev, 0)
        return (c + self.smoothing) / (ctx + self.smoothing * V)
    
    def trans(self, prev, curr):
        p = self.lam * self._p_bi(prev, curr) + (1 - self.lam) * self._p_uni(curr)
        return math.log(max(p, 1e-15))
    
    def emit(self, surface, state):
        surface = surface.lower()
        if surface not in self.emission_counts:
            return 0.0
        V = len(self.states)
        c = self.emission_counts[surface].get(state, 0)
        t = self.surface_totals[surface]
        return math.log(max((c + self.smoothing) / (t + self.smoothing * V), 1e-15))
    
    def disambiguate(self, surfaces, candidate_lists):
        n = len(candidate_lists)
        if n == 0:
            return []
        best = [{} for _ in range(n)]
        back = [{} for _ in range(n)]
        
        # t=0
        for i, fst in enumerate(candidate_lists[0]):
            s = extract_disambiguator_state(fst)
            if s == 'PUNCT':
                best[0][i] = 0.0
            else:
                best[0][i] = self.trans('START', s) + self.beta * self.emit(surfaces[0], s)
        
        # t>0: score = prev + trans(prev_s, curr_s) + beta * emit(w, curr_s)
        for t in range(1, n):
            for i, curr_fst in enumerate(candidate_lists[t]):
                curr_s = extract_disambiguator_state(curr_fst)
                if curr_s == 'PUNCT':
                    best_j = max(best[t-1], key=best[t-1].get) if best[t-1] else 0
                    best[t][i] = best[t-1].get(best_j, 0.0)
                    back[t][i] = best_j
                    continue
                max_score = float('-inf')
                best_j = 0
                for j, prev_fst in enumerate(candidate_lists[t-1]):
                    prev_s = extract_disambiguator_state(prev_fst)
                    if prev_s == 'PUNCT':
                        prev_s = 'START'
                    score = best[t-1][j] + self.trans(prev_s, curr_s) + self.beta * self.emit(surfaces[t], curr_s)
                    if score > max_score:
                        max_score = score
                        best_j = j
                best[t][i] = max_score
                back[t][i] = best_j
        
        if not best[n-1]:
            return [c[0] for c in candidate_lists]
        idx = max(best[n-1], key=best[n-1].get)
        path = []
        for t in range(n-1, -1, -1):
            path.append(candidate_lists[t][idx])
            idx = back[t].get(idx, 0) if t > 0 else 0
        return list(reversed(path))

print('TunableHMMDisambiguator hazir.')


In [ ]:
# ============================================================
# BOLUM 19: ACCURACY OLCUMU + GRID SWEEP
# ============================================================
# Gold standard: UD mapped veri (tek candidate = kesin cevap)
# Simulated candidates: gold + rastgele diger state'ler

import random
import math

random.seed(42)

def create_eval_set(mapped_data, n_distractors=2):
    # Her token icin:
    #   - gold FST analizi
    #   - + n_distractors rastgele yanlis candidate
    # Cumle bazinda grupla
    
    all_fst = [t['fst_analysis'] for t in mapped_data if 'PUNCT' not in t['fst_analysis']]
    
    sentences = []  # list of (surfaces, candidate_lists, gold_indices)
    curr_surfaces = []
    curr_candidates = []
    curr_golds = []
    
    for tok in mapped_data:
        fst = tok['fst_analysis']
        surface = tok['form'].lower()
        
        if 'PUNCT' in fst:
            if curr_surfaces:
                sentences.append((curr_surfaces, curr_candidates, curr_golds))
            curr_surfaces = []
            curr_candidates = []
            curr_golds = []
            continue
        
        # Candidate listesi: gold + distractors
        candidates = [fst]  # gold ilk sirada
        gold_idx = 0
        
        # Distractor: rastgele baska FST analizleri (ayni lemma, farkli state)
        lemma = fst.split('+')[0] if '+' in fst else fst
        gold_state = extract_disambiguator_state(fst)
        
        distractors_added = 0
        attempts = 0
        while distractors_added < n_distractors and attempts < 20:
            alt = random.choice(all_fst)
            alt_state = extract_disambiguator_state(alt)
            if alt_state != gold_state:  # Farkli state olmali
                alt_fst = lemma + '+' + alt_state  # Ayni lemma, farkli pattern
                if alt_fst not in candidates:
                    candidates.append(alt_fst)
                    distractors_added += 1
            attempts += 1
        
        # Shuffle (gold pozisyonunu kaydet)
        indexed = list(enumerate(candidates))
        random.shuffle(indexed)
        new_gold_idx = [new_i for new_i, (orig_i, _) in enumerate(indexed) if orig_i == 0][0]
        shuffled_cands = [c for _, c in indexed]
        
        curr_surfaces.append(surface)
        curr_candidates.append(shuffled_cands)
        curr_golds.append(new_gold_idx)
    
    if curr_surfaces:
        sentences.append((curr_surfaces, curr_candidates, curr_golds))
    
    return sentences


def eval_accuracy(model, eval_set):
    correct = 0
    total = 0
    
    for surfaces, candidate_lists, gold_indices in eval_set:
        predictions = model.disambiguate(surfaces, candidate_lists)
        
        for t, (pred, cands, gold_idx) in enumerate(zip(predictions, candidate_lists, gold_indices)):
            gold_fst = cands[gold_idx]
            total += 1
            if pred == gold_fst:
                correct += 1
    
    return correct / max(total, 1), correct, total


# ── Eval set olustur ──
eval_set = create_eval_set(mapped_data, n_distractors=2)
total_sents = len(eval_set)
total_tokens = sum(len(s) for s, _, _ in eval_set)
print(f'Eval set: {total_sents} cumle, {total_tokens} token')
print(f'Her token: 1 gold + 2 distractor = 3 candidate\n')

# ── GRID SWEEP ──
lambdas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
betas = [0.5, 1.0, 1.5, 2.0]

print(f'GRID SWEEP: lambda x beta')
print(f'={"=" * 65}')

# Header
header = f'{"":<8}'
for b in betas:
    header += f'  b={b:<5}'
print(header)
print('-' * 40)

best_acc = 0
best_config = (0, 0)
results_grid = {}

for lam in lambdas:
    row = f'l={lam:<5}'
    for beta in betas:
        model = TunableHMMDisambiguator(lam=lam, beta=beta, smoothing=0.01)
        model.train(mapped_data)
        acc, cor, tot = eval_accuracy(model, eval_set)
        row += f'  {acc*100:>5.1f}%'
        results_grid[(lam, beta)] = acc
        if acc > best_acc:
            best_acc = acc
            best_config = (lam, beta)
    print(row)

print(f'\nEN IYI: lambda={best_config[0]}, beta={best_config[1]} -> {best_acc*100:.1f}%')


In [ ]:
# ============================================================
# BOLUM 20: EN IYI MODEL ILE SON TEST + KAYIT
# ============================================================

best_lam, best_beta = best_config

final_model = TunableHMMDisambiguator(lam=best_lam, beta=best_beta, smoothing=0.01)
final_model.train(mapped_data)

acc, cor, tot = eval_accuracy(final_model, eval_set)

print(f'FINAL MODEL')
print(f'={"=" * 50}')
print(f'Lambda:   {best_lam}')
print(f'Beta:     {best_beta}')
print(f'Accuracy: {cor}/{tot} ({acc*100:.1f}%)')
print(f'States:   {len(final_model.states)}')
print(f'Bigrams:  {len(final_model.bigram_counts)}')
print(f'Surfaces: {len(final_model.emission_counts)}')

# ── Cumle bazinda test ──
print(f'\nORNEK CUMLELER:')
print('-' * 70)

for sent_i, (surfaces, cand_lists, golds) in enumerate(eval_set[:5]):
    preds = final_model.disambiguate(surfaces, cand_lists)
    print(f'\nCumle {sent_i+1}: {" ".join(surfaces)}')
    for t, (surf, pred, cands, g_idx) in enumerate(zip(surfaces, preds, cand_lists, golds)):
        gold = cands[g_idx]
        match = 'OK' if pred == gold else 'XX'
        pred_s = extract_disambiguator_state(pred)
        gold_s = extract_disambiguator_state(gold)
        n_c = len(cands)
        print(f'  [{match}] {surf:<15} pred={pred_s:<20} gold={gold_s:<20} ({n_c} cand)')

# ── Kaydet ──
final_export = {
    'metadata': {
        'model_type': 'Interpolated Bigram HMM with conditional lexical emission',
        'lambda': best_lam,
        'beta': best_beta,
        'smoothing': 0.01,
        'accuracy': acc,
        'total_tokens': final_model.total_tokens,
        'states': len(final_model.states),
        'bigrams': len(final_model.bigram_counts),
        'surfaces': len(final_model.emission_counts),
    },
    'unigrams': dict(final_model.unigram_counts),
    'bigrams': {
        f'{p}|||{c}': n for (p,c),n in final_model.bigram_counts.items()
    },
    'emissions': {
        s: dict(d) for s,d in final_model.emission_counts.items()
    },
    'states': sorted(list(final_model.states)),
}

with open('final_hmm_model.json', 'w', encoding='utf-8') as f:
    json.dump(final_export, f, ensure_ascii=False, indent=2)

print(f'\nModel kaydedildi: final_hmm_model.json')
print(f'Grid sonuclari:')
for (l, b), a in sorted(results_grid.items(), key=lambda x: -x[1])[:5]:
    marker = ' <-- BEST' if (l,b) == best_config else ''
    print(f'  l={l}, b={b} -> {a*100:.1f}%{marker}')


## DUZELTILMIS EVALUATION

Onceki eval neden yanlis:
- Distractor'lar random state sampling -> yapay kolay
- Train = Eval -> memorization / leakage
- %99.7 gercek disambiguation performansi degil

Yeni eval:
1. **Sentence-level train/test split** (80/20)
2. **Surface-conditioned candidates**: Ayni surface form icin training'de gorulen TUM state'ler
3. **Overfitting testi**: Gold'u random ile degistir, accuracy dusmeli
4. **Gercek ambiguity**: `guzel` -> ADJ veya NOUN (training'den ogrenilmis)


In [ ]:
# ============================================================
# BOLUM 21: GERCEK EVALUATION SETUP
# ============================================================

import random
import math
from collections import defaultdict, Counter

random.seed(42)

# ── 1. Cumle bazli train/test split ──

def split_into_sentences(mapped_data):
    sentences = []
    curr = []
    for tok in mapped_data:
        if 'PUNCT' in tok['fst_analysis']:
            if curr:
                sentences.append(curr)
                curr = []
        else:
            curr.append(tok)
    if curr:
        sentences.append(curr)
    return sentences

sentences = split_into_sentences(mapped_data)
random.shuffle(sentences)

split_idx = int(len(sentences) * 0.8)
train_sents = sentences[:split_idx]
test_sents = sentences[split_idx:]

train_data = [tok for sent in train_sents for tok in sent]
test_data = [tok for sent in test_sents for tok in sent]

print(f'Cumle bazli split:')
print(f'  Train: {len(train_sents)} cumle, {len(train_data)} token')
print(f'  Test:  {len(test_sents)} cumle, {len(test_data)} token')

# Surface overlap kontrolu
train_surfaces = set(t['form'].lower() for t in train_data)
test_surfaces = set(t['form'].lower() for t in test_data)
overlap = train_surfaces & test_surfaces
test_oov = test_surfaces - train_surfaces
print(f'  Surface overlap: {len(overlap)} / {len(test_surfaces)} ({100*len(overlap)/max(len(test_surfaces),1):.0f}%)')
print(f'  Test OOV: {len(test_oov)} surface ({100*len(test_oov)/max(len(test_surfaces),1):.0f}%)')


In [ ]:
# ============================================================
# BOLUM 22: SURFACE-CONDITIONED CANDIDATE URETIMI
# ============================================================
# Gercek ambiguity: Bir surface form training'de hangi state'lerle
# gorulmusse, o state'ler candidate olur.

# Training'den surface -> state haritasi olustur
surface_to_states = defaultdict(set)   # surface -> {state1, state2, ...}
surface_to_fst = defaultdict(dict)     # surface -> {state: fst_analysis}

for tok in train_data:
    surface = tok['form'].lower()
    fst = tok['fst_analysis']
    state = extract_disambiguator_state(fst)
    surface_to_states[surface].add(state)
    surface_to_fst[surface][state] = fst

# Ambiguity istatistikleri
ambiguity_dist = Counter()
for surf, states in surface_to_states.items():
    ambiguity_dist[len(states)] += 1

print('SURFACE AMBIGUITY (Training Verisi)')
print('=' * 50)
total_surf = sum(ambiguity_dist.values())
for n_states, count in sorted(ambiguity_dist.items()):
    pct = 100 * count / total_surf
    bar = '#' * int(pct / 2)
    print(f'  {n_states} state: {count:>5} surface ({pct:>5.1f}%) {bar}')

# Ambiguous surface ornekleri
print(f'\nAmbiguous Surface Ornekleri (2+ state):')
count = 0
for surf, states in sorted(surface_to_states.items()):
    if len(states) >= 2:
        print(f'  {surf:<20} -> {sorted(states)}')
        count += 1
        if count >= 15:
            break

# ── Test set icin candidate listesi olustur ──
def build_real_eval(test_sents, surface_to_states, surface_to_fst):
    eval_set = []  # list of (surfaces, candidate_lists, gold_indices)
    
    stats = {'total': 0, 'ambiguous': 0, 'unambiguous': 0, 'unseen': 0}
    
    for sent in test_sents:
        surfaces = []
        candidate_lists = []
        gold_indices = []
        
        for tok in sent:
            surface = tok['form'].lower()
            gold_fst = tok['fst_analysis']
            gold_state = extract_disambiguator_state(gold_fst)
            
            stats['total'] += 1
            
            if surface in surface_to_states:
                known_states = surface_to_states[surface]
                
                # Candidate listesi: training'de gorulen tum state'ler
                candidates = []
                gold_idx = -1
                
                for state in sorted(known_states):
                    # Lemma'yi koruyarak candidate olustur
                    lemma = gold_fst.split('+')[0] if '+' in gold_fst else gold_fst
                    if state in surface_to_fst[surface]:
                        cand_fst = surface_to_fst[surface][state]
                    else:
                        cand_fst = lemma + '+' + state
                    candidates.append(cand_fst)
                    if state == gold_state:
                        gold_idx = len(candidates) - 1
                
                # Gold state training'de gorulmemis olabilir
                if gold_idx == -1:
                    candidates.append(gold_fst)
                    gold_idx = len(candidates) - 1
                
                if len(candidates) > 1:
                    stats['ambiguous'] += 1
                else:
                    stats['unambiguous'] += 1
            else:
                # OOV: sadece gold
                candidates = [gold_fst]
                gold_idx = 0
                stats['unseen'] += 1
            
            surfaces.append(surface)
            candidate_lists.append(candidates)
            gold_indices.append(gold_idx)
        
        if surfaces:
            eval_set.append((surfaces, candidate_lists, gold_indices))
    
    return eval_set, stats

real_eval, eval_stats = build_real_eval(test_sents, surface_to_states, surface_to_fst)

print(f'\nTEST SET ISTATISTIKLERI')
print(f'  Toplam token: {eval_stats["total"]}')
print(f'  Ambiguous:    {eval_stats["ambiguous"]} ({100*eval_stats["ambiguous"]/max(eval_stats["total"],1):.1f}%)')
print(f'  Unambiguous:  {eval_stats["unambiguous"]} ({100*eval_stats["unambiguous"]/max(eval_stats["total"],1):.1f}%)')
print(f'  OOV/unseen:   {eval_stats["unseen"]} ({100*eval_stats["unseen"]/max(eval_stats["total"],1):.1f}%)')


In [ ]:
# ============================================================
# BOLUM 23: GERCEK ACCURACY + GRID SWEEP
# ============================================================

def eval_real_accuracy(model, eval_set, detail=False):
    correct = 0
    total = 0
    ambig_correct = 0
    ambig_total = 0
    errors = []
    
    for surfaces, cand_lists, golds in eval_set:
        preds = model.disambiguate(surfaces, cand_lists)
        
        for t, (pred, cands, g_idx) in enumerate(zip(preds, cand_lists, golds)):
            gold = cands[g_idx]
            total += 1
            is_ambig = len(cands) > 1
            
            if pred == gold:
                correct += 1
                if is_ambig:
                    ambig_correct += 1
            else:
                if is_ambig and len(errors) < 30:
                    errors.append({
                        'surface': surfaces[t],
                        'gold': extract_disambiguator_state(gold),
                        'pred': extract_disambiguator_state(pred),
                        'n_cands': len(cands),
                    })
            
            if is_ambig:
                ambig_total += 1
    
    overall = correct / max(total, 1)
    ambig_acc = ambig_correct / max(ambig_total, 1)
    
    return {
        'overall': overall,
        'overall_n': (correct, total),
        'ambiguous_acc': ambig_acc,
        'ambiguous_n': (ambig_correct, ambig_total),
        'errors': errors,
    }

# ── GRID SWEEP ──
lambdas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
betas = [0.5, 1.0, 1.5, 2.0]

print('GRID SWEEP (Train/Test Split, Surface-Conditioned Candidates)')
print('=' * 70)
print(f'Train: {len(train_data)} tok | Test: {len(test_data)} tok | Ambig: {eval_stats["ambiguous"]} tok')
print()

# Overall accuracy table
header = f'{"":<8}'
for b in betas:
    header += f'  b={b:<6}'
print('OVERALL ACCURACY:')
print(header)
print('-' * 45)

best_overall = 0
best_ambig = 0
best_config_overall = (0, 0)
best_config_ambig = (0, 0)
all_results = {}

for lam in lambdas:
    row_overall = f'l={lam:<5}'
    for beta in betas:
        model = TunableHMMDisambiguator(lam=lam, beta=beta, smoothing=0.01)
        model.train(train_data)  # SADECE TRAIN ILE EGIT
        res = eval_real_accuracy(model, real_eval)
        
        row_overall += f'  {res["overall"]*100:>5.1f}%'
        all_results[(lam, beta)] = res
        
        if res['overall'] > best_overall:
            best_overall = res['overall']
            best_config_overall = (lam, beta)
        if res['ambiguous_acc'] > best_ambig:
            best_ambig = res['ambiguous_acc']
            best_config_ambig = (lam, beta)
    print(row_overall)

# Ambiguous accuracy table
print(f'\nAMBIGUOUS-ONLY ACCURACY:')
print(header)
print('-' * 45)
for lam in lambdas:
    row = f'l={lam:<5}'
    for beta in betas:
        res = all_results[(lam, beta)]
        row += f'  {res["ambiguous_acc"]*100:>5.1f}%'
    print(row)

print(f'\nEN IYI OVERALL:  l={best_config_overall[0]} b={best_config_overall[1]} -> {best_overall*100:.1f}%')
print(f'EN IYI AMBIG:    l={best_config_ambig[0]} b={best_config_ambig[1]} -> {best_ambig*100:.1f}%')


In [ ]:
# ============================================================
# BOLUM 24: HATA ANALIZI + OVERFITTING TESTI
# ============================================================

# ── En iyi modelin hata analizi ──
bl, bb = best_config_overall
best_model = TunableHMMDisambiguator(lam=bl, beta=bb, smoothing=0.01)
best_model.train(train_data)
best_res = eval_real_accuracy(best_model, real_eval, detail=True)

print(f'HATA ANALIZI (l={bl}, b={bb})')
print(f'Overall: {best_res["overall_n"][0]}/{best_res["overall_n"][1]} ({best_res["overall"]*100:.1f}%)')
print(f'Ambiguous: {best_res["ambiguous_n"][0]}/{best_res["ambiguous_n"][1]} ({best_res["ambiguous_acc"]*100:.1f}%)')

# Hatalar
if best_res['errors']:
    print(f'\nIlk 20 Hata (sadece ambiguous):')
    print(f'{"Surface":<20} {"Gold":<25} {"Predicted":<25} {"#C"}')
    print('-' * 75)
    for err in best_res['errors'][:20]:
        print(f'{err["surface"]:<20} {err["gold"]:<25} {err["pred"]:<25} {err["n_cands"]}')
    
    # Hata pattern'leri
    err_patterns = Counter()
    for err in best_res['errors']:
        err_patterns[f'{err["gold"]} -> {err["pred"]}'] += 1
    print(f'\nEn Sik Hata Pattern\'leri:')
    for pattern, count in err_patterns.most_common(10):
        print(f'  {count:>3}x  {pattern}')
else:
    print(f'Hata yok.')

# ── OVERFITTING TESTI ──
print(f'\n{"=" * 60}')
print(f'OVERFITTING TESTI')
print(f'={"=" * 59}')

# Train set ile eval et (leakage testi)
train_eval, _ = build_real_eval(train_sents, surface_to_states, surface_to_fst)
train_res = eval_real_accuracy(best_model, train_eval)

print(f'Train accuracy: {train_res["overall"]*100:.1f}% ({train_res["overall_n"][0]}/{train_res["overall_n"][1]})')
print(f'Test accuracy:  {best_res["overall"]*100:.1f}% ({best_res["overall_n"][0]}/{best_res["overall_n"][1]})')
gap = train_res['overall'] - best_res['overall']
print(f'Gap:            {gap*100:.1f}%')

if gap > 0.10:
    print(f'UYARI: Yuksek overfitting gabi ({gap*100:.1f}% > 10%)')
elif gap > 0.05:
    print(f'DIKKAT: Orta overfitting ({gap*100:.1f}%)')
else:
    print(f'OK: Dusuk overfitting ({gap*100:.1f}%)')

# Ambiguous bazinda
print(f'\nTrain ambig acc: {train_res["ambiguous_acc"]*100:.1f}% ({train_res["ambiguous_n"][0]}/{train_res["ambiguous_n"][1]})')
print(f'Test ambig acc:  {best_res["ambiguous_acc"]*100:.1f}% ({best_res["ambiguous_n"][0]}/{best_res["ambiguous_n"][1]})')

# ── Random gold testi ──
print(f'\nRANDOM GOLD TESTI (gold yerine rastgele candidate sec):')
import copy
random_eval = []
for surfaces, cands, golds in real_eval:
    fake_golds = []
    for g, c in zip(golds, cands):
        if len(c) > 1:
            # Gold OLMAYAN bir candidate sec
            alternatives = [i for i in range(len(c)) if i != g]
            fake_golds.append(random.choice(alternatives) if alternatives else g)
        else:
            fake_golds.append(g)
    random_eval.append((surfaces, cands, fake_golds))

random_res = eval_real_accuracy(best_model, random_eval)
print(f'Random-gold accuracy: {random_res["overall"]*100:.1f}%')
print(f'Gercek accuracy:      {best_res["overall"]*100:.1f}%')
diff = best_res['overall'] - random_res['overall']
print(f'Fark: {diff*100:.1f}%')
if diff > 0.1:
    print(f'Model gercekten ayirt edebiliyor (fark > 10%)')
else:
    print(f'UYARI: Model yeterince ayirt edemiyor')
